In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_1(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_2(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_3(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_4(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
def NN_model_structure_regression_5(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [23]:
def NN_model_structure_regression_6(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [24]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [25]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_1(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 411us/step - lo

1052/1052 [==============================] - 0s 92us/step - loss: 21.8810 - val_loss: 26.0039
Epoch 66/1000
1052/1052 [==============================] - 0s 98us/step - loss: 22.1261 - val_loss: 25.4663
Epoch 67/1000
1052/1052 [==============================] - 0s 96us/step - loss: 21.8649 - val_loss: 25.3604
Epoch 68/1000
1052/1052 [==============================] - 0s 100us/step - loss: 21.7162 - val_loss: 25.5009
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 21.9001 - val_loss: 25.6936
Epoch 70/1000
1052/1052 [==============================] - 0s 100us/step - loss: 22.0035 - val_loss: 26.1389
Epoch 71/1000
1052/1052 [==============================] - 0s 96us/step - loss: 22.0356 - val_loss: 26.0249
Epoch 72/1000
1052/1052 [==============================] - 0s 95us/step - loss: 22.0018 - val_loss: 25.9081
Epoch 73/1000
1052/1052 [==============================] - 0s 100us/step - loss: 21.8790 - val_loss: 26.2634
Epoch 74/1000
1052/1052 [==============

1052/1052 [==============================] - 0s 88us/step - loss: 22.2437 - val_loss: 26.7113
Epoch 141/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.8637 - val_loss: 26.2607
Epoch 142/1000
1052/1052 [==============================] - 0s 84us/step - loss: 21.5677 - val_loss: 25.6067
Epoch 143/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.2219 - val_loss: 26.2450
Epoch 144/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.4557 - val_loss: 25.6124
Epoch 145/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.5848 - val_loss: 26.6953
Epoch 146/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.7013 - val_loss: 26.0434
Epoch 147/1000
1052/1052 [==============================] - 0s 84us/step - loss: 22.2140 - val_loss: 25.9594
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.5475 - val_loss: 25.2302
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 16.6407 - val_loss: 18.8315
Epoch 216/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.0008 - val_loss: 18.9548
Epoch 217/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.7312 - val_loss: 19.4881
Epoch 218/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.4916 - val_loss: 18.6400
Epoch 219/1000
1052/1052 [==============================] - 0s 90us/step - loss: 16.1571 - val_loss: 19.7297
Epoch 220/1000
1052/1052 [==============================] - 0s 85us/step - loss: 16.3939 - val_loss: 18.9357
Epoch 221/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.4312 - val_loss: 19.8506
Epoch 222/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.3919 - val_loss: 19.0298
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 16.1944 - val_loss: 18.5236
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 92us/step - loss: 12.1017 - val_loss: 15.3037
Epoch 291/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.1053 - val_loss: 15.6009
Epoch 292/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.5652 - val_loss: 17.1044
Epoch 293/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.1199 - val_loss: 18.9269
Epoch 294/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.3455 - val_loss: 16.8889
Epoch 295/1000
1052/1052 [==============================] - 0s 90us/step - loss: 12.2363 - val_loss: 14.6605
Epoch 296/1000
1052/1052 [==============================] - 0s 93us/step - loss: 11.9010 - val_loss: 15.5539
Epoch 297/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.8441 - val_loss: 14.6973
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.9165 - val_loss: 16.3392
Epoch 299/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 96us/step - loss: 8.1891 - val_loss: 9.2661
Epoch 745/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.2263 - val_loss: 9.3928
Epoch 746/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.1472 - val_loss: 9.9285
Epoch 747/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.1448 - val_loss: 12.0750
Epoch 748/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.2488 - val_loss: 9.5808
Epoch 749/1000
1052/1052 [==============================] - 0s 111us/step - loss: 8.2937 - val_loss: 9.6129
Epoch 750/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2728 - val_loss: 9.6125
Epoch 751/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3862 - val_loss: 9.6877
Epoch 752/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7033 - val_loss: 9.4924
Epoch 753/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 89us/step - loss: 8.0481 - val_loss: 9.4408
Epoch 821/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3517 - val_loss: 10.8233
Epoch 822/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2718 - val_loss: 10.0813
Epoch 823/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3892 - val_loss: 11.0176
Epoch 824/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6137 - val_loss: 9.1332
Epoch 825/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2417 - val_loss: 9.0923
Epoch 826/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1576 - val_loss: 9.7847
Epoch 827/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.1163 - val_loss: 11.0568
Epoch 828/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5663 - val_loss: 9.8979
Epoch 829/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 90us/step - loss: 8.4679 - val_loss: 11.2795
Epoch 897/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5065 - val_loss: 9.3782
Epoch 898/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2968 - val_loss: 10.7718
Epoch 899/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.9887 - val_loss: 10.5682
Epoch 900/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.0801 - val_loss: 11.1308
Epoch 901/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.0116 - val_loss: 9.2057
Epoch 902/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6546 - val_loss: 9.3449
Epoch 903/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3460 - val_loss: 9.8453
Epoch 904/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1612 - val_loss: 9.9048
Epoch 905/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 90us/step - loss: 8.7846 - val_loss: 10.6231
Epoch 973/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3537 - val_loss: 9.6327
Epoch 974/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2696 - val_loss: 9.7711
Epoch 975/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2722 - val_loss: 9.1773
Epoch 976/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.9821 - val_loss: 9.5210
Epoch 977/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2838 - val_loss: 9.9615
Epoch 978/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5899 - val_loss: 8.9906
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0543 - val_loss: 8.9369
Epoch 980/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1631 - val_loss: 9.8080
Epoch 981/1000
1052/1052 [=========================

[array([[-0.68602866, -3.135599  , -0.16234018, -1.7945377 ,  4.101883  ],
        [-1.3956003 ,  0.08365555, -0.20418075,  0.48436937, -0.23398285],
        [-1.5491245 , -0.02433117, -0.2863726 ,  0.42261127,  0.7450873 ],
        [ 0.26079416,  0.06222427,  0.126999  ,  0.03821959, -0.15199816],
        [ 0.24986778, -2.3047254 , -0.18715408, -0.31507376,  0.3466104 ]],
       dtype=float32),
 array([-0.6757135, -4.704725 ,  1.4873314, -1.2968112,  4.9995384],
       dtype=float32),
 array([[-0.5912905 ,  0.9639903 ,  0.86775905, -0.18733127, -0.1524617 ],
        [ 2.0874176 , -2.7747774 , -3.0484421 ,  2.0541475 , -2.9543111 ],
        [-2.8790624 ,  1.9575367 ,  2.8257475 , -2.3929565 ,  2.4055622 ],
        [-1.2688606 ,  0.41023287,  0.97238207, -0.63373667,  0.77693576],
        [-2.1403685 ,  3.027027  ,  2.1291656 , -2.964412  ,  2.6085274 ]],
       dtype=float32),
 array([-2.3099806,  2.3472826,  2.4673135, -2.244411 ,  2.2594266],
       dtype=float32),
 array([[-2.585063

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure1_4th.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_2(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 195us/step - l

1052/1052 [==============================] - 0s 85us/step - loss: 15.7471 - val_loss: 18.2393
Epoch 68/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.4787 - val_loss: 18.5063
Epoch 69/1000
1052/1052 [==============================] - 0s 88us/step - loss: 15.3653 - val_loss: 19.0901
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 15.1842 - val_loss: 18.7734
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 15.3267 - val_loss: 18.0495
Epoch 72/1000
1052/1052 [==============================] - 0s 99us/step - loss: 15.3618 - val_loss: 18.2890
Epoch 73/1000
1052/1052 [==============================] - 0s 92us/step - loss: 15.1271 - val_loss: 18.4169
Epoch 74/1000
1052/1052 [==============================] - 0s 94us/step - loss: 15.2676 - val_loss: 19.0929
Epoch 75/1000
1052/1052 [==============================] - 0s 96us/step - loss: 14.9066 - val_loss: 17.5339
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 87us/step - loss: 11.6484 - val_loss: 12.9908
Epoch 143/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.4779 - val_loss: 15.9037
Epoch 144/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.7216 - val_loss: 13.2426
Epoch 145/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.3813 - val_loss: 12.4448
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.7859 - val_loss: 14.3341
Epoch 147/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.7288 - val_loss: 15.0543
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.9901 - val_loss: 13.5901
Epoch 149/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.6112 - val_loss: 13.1740
Epoch 150/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.3807 - val_loss: 12.9334
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 9.5178 - val_loss: 11.4973
Epoch 218/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.4480 - val_loss: 13.2823
Epoch 219/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.5618 - val_loss: 10.9608
Epoch 220/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.4135 - val_loss: 11.0736
Epoch 221/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.1300 - val_loss: 13.5150
Epoch 222/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.0814 - val_loss: 11.5033
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.5330 - val_loss: 11.4459
Epoch 224/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.3754 - val_loss: 12.2857
Epoch 225/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.5458 - val_loss: 15.6748
Epoch 226/1000
1052/1052 [=================

Epoch 293/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9102 - val_loss: 11.2765
Epoch 294/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.1810 - val_loss: 11.1563
Epoch 295/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.4448 - val_loss: 11.2215
Epoch 296/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.1650 - val_loss: 11.7158
Epoch 297/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.9375 - val_loss: 11.5928
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.2763 - val_loss: 11.2640
Epoch 299/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.6485 - val_loss: 12.0461
Epoch 300/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.7013 - val_loss: 11.2425
Epoch 301/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9811 - val_loss: 11.4360
Epoch 302/1000
1052/1052 [==

...

1052/1052 [==============================] - 0s 85us/step - loss: 8.3617 - val_loss: 9.1165
Epoch 750/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2906 - val_loss: 9.6808
Epoch 751/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1830 - val_loss: 9.4161
Epoch 752/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6343 - val_loss: 9.6927
Epoch 753/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.4157 - val_loss: 13.7225
Epoch 754/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2209 - val_loss: 9.8089
Epoch 755/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.2096 - val_loss: 12.8379
Epoch 756/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.4555 - val_loss: 9.1153
Epoch 757/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.0130 - val_loss: 11.2213
Epoch 758/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 86us/step - loss: 8.6617 - val_loss: 9.7769
Epoch 826/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1694 - val_loss: 9.8331
Epoch 827/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1680 - val_loss: 9.5390
Epoch 828/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.5463 - val_loss: 9.5992
Epoch 829/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3404 - val_loss: 9.7967
Epoch 830/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5181 - val_loss: 9.8086
Epoch 831/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.1175 - val_loss: 9.9148
Epoch 832/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1622 - val_loss: 9.3619
Epoch 833/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8671 - val_loss: 9.3495
Epoch 834/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 84us/step - loss: 8.2337 - val_loss: 10.2266
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4911 - val_loss: 9.9139
Epoch 903/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2379 - val_loss: 11.5437
Epoch 904/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.2554 - val_loss: 9.9170
Epoch 905/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.0449 - val_loss: 9.9193
Epoch 906/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2016 - val_loss: 9.4146
Epoch 907/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.2462 - val_loss: 11.7444
Epoch 908/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7076 - val_loss: 9.8245
Epoch 909/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.8923 - val_loss: 9.2174
Epoch 910/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 89us/step - loss: 8.1180 - val_loss: 9.6552
Epoch 978/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.2790 - val_loss: 9.7953
Epoch 979/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.2290 - val_loss: 9.4941
Epoch 980/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.4225 - val_loss: 10.8231
Epoch 981/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6226 - val_loss: 9.3780
Epoch 982/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.9895 - val_loss: 9.0464
Epoch 983/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2023 - val_loss: 11.7077
Epoch 984/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.6229 - val_loss: 9.9265
Epoch 985/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.0503 - val_loss: 9.3989
Epoch 986/1000
1052/1052 [========================

[array([[-1.8543719 ,  3.994964  , -0.67607903,  3.3079267 , -0.14229144],
        [ 0.47271243, -0.21911685, -1.3893162 , -0.07347753, -0.19800112],
        [ 0.3873367 ,  0.7414873 , -1.5729681 , -0.00752882, -0.29400936],
        [ 0.0279171 , -0.17475185,  0.24542645, -0.06149895,  0.12112733],
        [-0.29920578,  0.32019627,  0.28489867,  2.546552  , -0.18457963]],
       dtype=float32),
 array([-1.2868559,  4.9410443, -0.6747812,  4.751049 ,  1.3849336],
       dtype=float32),
 array([[-0.31863678, -0.07661062, -0.6605434 ,  0.2682548 ,  0.5082065 ,
          0.94094855, -0.62507105, -0.62306714, -1.0552505 ,  0.30977872],
        [-2.4209397 , -1.3607966 , -2.3481767 ,  1.5450659 ,  2.557503  ,
          2.1143527 , -1.9646051 , -2.5989234 , -2.2126536 ,  1.6150489 ],
        [-0.20090845,  0.25121745, -0.64416987,  0.55724216,  0.80736554,
         -0.16266158, -0.6740566 ,  0.34516183, -0.7820709 , -0.052527  ],
        [-1.1974512 , -0.5985419 , -1.1082377 ,  0.90537584,  

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure2_4th.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_3(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                90        
_________________________________________________________________
activation_6 (Activation)    (None, 15)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 16        
Total params: 136
Trainable params: 136
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 206us/step - l

1052/1052 [==============================] - 0s 86us/step - loss: 17.7074 - val_loss: 21.9960
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.7566 - val_loss: 22.6481
Epoch 69/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.0430 - val_loss: 22.9468
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.6414 - val_loss: 22.0133
Epoch 71/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.6613 - val_loss: 21.3819
Epoch 72/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.0294 - val_loss: 23.2146
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.8521 - val_loss: 21.9729
Epoch 74/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.3203 - val_loss: 20.4871
Epoch 75/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.0795 - val_loss: 23.2516
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 93us/step - loss: 11.4353 - val_loss: 14.6221
Epoch 143/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.2318 - val_loss: 13.7670
Epoch 144/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.7544 - val_loss: 14.0214
Epoch 145/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.2072 - val_loss: 15.0848
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.5073 - val_loss: 14.8886
Epoch 147/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.1314 - val_loss: 13.5594
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.5037 - val_loss: 14.0679
Epoch 149/1000
1052/1052 [==============================] - 0s 91us/step - loss: 10.8905 - val_loss: 13.6491
Epoch 150/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.2400 - val_loss: 12.9712
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 86us/step - loss: 9.3622 - val_loss: 10.8644
Epoch 218/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.3920 - val_loss: 10.5584
Epoch 219/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.0639 - val_loss: 11.3435
Epoch 220/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.1144 - val_loss: 11.9016
Epoch 221/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.6076 - val_loss: 11.3290
Epoch 222/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.3030 - val_loss: 12.3334
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0520 - val_loss: 11.0178
Epoch 224/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.8842 - val_loss: 11.4534
Epoch 225/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.7146 - val_loss: 13.3094
Epoch 226/1000
1052/1052 [================

Epoch 293/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.6128 - val_loss: 10.6045
Epoch 294/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.8200 - val_loss: 11.7327
Epoch 295/1000
1052/1052 [==============================] - 0s 95us/step - loss: 9.0188 - val_loss: 10.3574
Epoch 296/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.7219 - val_loss: 10.5063
Epoch 297/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.1789 - val_loss: 10.5160
Epoch 298/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.9410 - val_loss: 10.1723
Epoch 299/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.6722 - val_loss: 10.1345
Epoch 300/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.6388 - val_loss: 10.1877
Epoch 301/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.9230 - val_loss: 10.2065
Epoch 302/1000
1052/1052 [==

...

1052/1052 [==============================] - 0s 75us/step - loss: 8.5609 - val_loss: 8.9863
Epoch 750/1000
1052/1052 [==============================] - 0s 101us/step - loss: 8.1214 - val_loss: 10.6584
Epoch 751/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.1994 - val_loss: 10.3074
Epoch 752/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.1838 - val_loss: 9.0213
Epoch 753/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.1948 - val_loss: 9.7516
Epoch 754/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4795 - val_loss: 8.9225
Epoch 755/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.0986 - val_loss: 8.8820
Epoch 756/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.5454 - val_loss: 9.8260
Epoch 757/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.0132 - val_loss: 9.7578
Epoch 758/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 87us/step - loss: 8.0806 - val_loss: 9.2065
Epoch 826/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1415 - val_loss: 10.3894
Epoch 827/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3565 - val_loss: 10.1855
Epoch 828/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1808 - val_loss: 8.5742
Epoch 829/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.2537 - val_loss: 9.7198
Epoch 830/1000
1052/1052 [==============================] - 0s 104us/step - loss: 8.7759 - val_loss: 12.1050
Epoch 831/1000
1052/1052 [==============================] - 0s 106us/step - loss: 7.9688 - val_loss: 8.9365
Epoch 832/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.5819 - val_loss: 9.1107
Epoch 833/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7929 - val_loss: 9.4799
Epoch 834/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 82us/step - loss: 8.1402 - val_loss: 9.2139
Epoch 902/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.1058 - val_loss: 9.9899
Epoch 903/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0396 - val_loss: 10.2065
Epoch 904/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2595 - val_loss: 9.0259
Epoch 905/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1155 - val_loss: 8.9287
Epoch 906/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2253 - val_loss: 8.8436
Epoch 907/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.3040 - val_loss: 8.7304
Epoch 908/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.8757 - val_loss: 8.7913
Epoch 909/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.7159 - val_loss: 8.4473
Epoch 910/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 84us/step - loss: 8.1935 - val_loss: 10.9166
Epoch 978/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.5937 - val_loss: 10.1048
Epoch 979/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0924 - val_loss: 8.4497
Epoch 980/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.8577 - val_loss: 8.7891
Epoch 981/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.8502 - val_loss: 8.6779
Epoch 982/1000
1052/1052 [==============================] - 0s 81us/step - loss: 7.6223 - val_loss: 8.7338
Epoch 983/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4895 - val_loss: 9.0511
Epoch 984/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.4015 - val_loss: 8.9742
Epoch 985/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.3597 - val_loss: 8.5650
Epoch 986/1000
1052/1052 [========================

[array([[-3.4302034 , -0.8061793 , -2.179475  , -0.28243268, -3.4678533 ],
        [ 0.07419473, -1.8572897 ,  0.4691619 , -0.18793742,  0.13517979],
        [ 0.01813842, -2.2457294 ,  0.35899064, -0.42275238, -0.81176287],
        [ 0.04165632,  0.3591742 ,  0.01018146,  0.17718554,  0.13762471],
        [-2.4569123 ,  0.83139926, -0.30249998, -0.37302366, -0.2756122 ]],
       dtype=float32),
 array([-4.873136  , -1.092576  , -1.3773726 ,  0.67471004, -4.6214347 ],
       dtype=float32),
 array([[-5.8510274e-01, -1.7874317e+00, -1.0559667e+00, -1.5869122e+00,
          1.0325084e+00, -1.0920452e+00,  1.3409933e+00, -1.3368695e+00,
          1.7452401e+00, -1.1981992e+00,  6.7222965e-01, -1.6194640e+00,
          1.3226418e+00,  7.4678904e-01, -1.6411501e+00],
        [ 4.4768420e-01, -9.2916481e-02,  6.0430057e-02,  5.5874336e-01,
          4.0502824e-02,  3.7706462e-01,  3.5663873e-02, -1.6109066e-01,
         -1.9389254e-01,  7.0198953e-01, -2.9571411e-01, -9.8940350e-02,
        

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure3_4th.h5')

In [32]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_4(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_8 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 235us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 95us/step - loss: 14.5839 - val_loss: 18.0702
Epoch 68/1000
1052/1052 [==============================] - 0s 93us/step - loss: 14.9027 - val_loss: 18.3838
Epoch 69/1000
1052/1052 [==============================] - 0s 84us/step - loss: 14.5523 - val_loss: 18.6571
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.5937 - val_loss: 17.9711
Epoch 71/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.5932 - val_loss: 18.1478
Epoch 72/1000
1052/1052 [==============================] - 0s 82us/step - loss: 14.4756 - val_loss: 17.3742
Epoch 73/1000
1052/1052 [==============================] - 0s 97us/step - loss: 14.5896 - val_loss: 17.9517
Epoch 74/1000
1052/1052 [==============================] - 0s 98us/step - loss: 14.3638 - val_loss: 17.3849
Epoch 75/1000
1052/1052 [==============================] - 0s 98us/step - loss: 14.3937 - val_loss: 18.6486
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 88us/step - loss: 9.3055 - val_loss: 11.4051
Epoch 143/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.3952 - val_loss: 12.3803
Epoch 144/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.1838 - val_loss: 11.0953
Epoch 145/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.7914 - val_loss: 12.3388
Epoch 146/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.4400 - val_loss: 10.9356
Epoch 147/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.1356 - val_loss: 11.2754
Epoch 148/1000
1052/1052 [==============================] - 0s 95us/step - loss: 9.3618 - val_loss: 12.6292
Epoch 149/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.3463 - val_loss: 11.1780
Epoch 150/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.9274 - val_loss: 11.0698
Epoch 151/1000
1052/1052 [=================

Epoch 218/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.4303 - val_loss: 9.7630
Epoch 219/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.8887 - val_loss: 10.9500
Epoch 220/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8669 - val_loss: 10.7538
Epoch 221/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.6400 - val_loss: 10.9118
Epoch 222/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.4967 - val_loss: 10.2848
Epoch 223/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4974 - val_loss: 10.1049
Epoch 224/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.5646 - val_loss: 11.3308
Epoch 225/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.8644 - val_loss: 11.1623
Epoch 226/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.6791 - val_loss: 11.4415
Epoch 227/1000
1052/1052 [===

1052/1052 [==============================] - 0s 81us/step - loss: 8.8104 - val_loss: 9.9410
Epoch 295/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2784 - val_loss: 11.7129
Epoch 296/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.2579 - val_loss: 11.3321
Epoch 297/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3914 - val_loss: 10.3814
Epoch 298/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2278 - val_loss: 9.6403
Epoch 299/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.4452 - val_loss: 9.6528
Epoch 300/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.3375 - val_loss: 9.9428
Epoch 301/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.0891 - val_loss: 10.0975
Epoch 302/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1811 - val_loss: 9.4832
Epoch 303/1000
1052/1052 [======================

...

1052/1052 [==============================] - 0s 85us/step - loss: 7.8313 - val_loss: 8.6961
Epoch 751/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.6571 - val_loss: 9.1699
Epoch 752/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.6971 - val_loss: 8.5429
Epoch 753/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.4728 - val_loss: 8.9249
Epoch 754/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.7931 - val_loss: 8.9870
Epoch 755/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.3457 - val_loss: 8.5020
Epoch 756/1000
1052/1052 [==============================] - 0s 96us/step - loss: 7.5271 - val_loss: 8.9644
Epoch 757/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.5217 - val_loss: 8.7571
Epoch 758/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.5212 - val_loss: 8.6879
Epoch 759/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 7.8501 - val_loss: 8.4268
Epoch 827/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0785 - val_loss: 11.2167
Epoch 828/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6034 - val_loss: 10.0389
Epoch 829/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.9385 - val_loss: 9.0621
Epoch 830/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.6003 - val_loss: 9.9951
Epoch 831/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.4094 - val_loss: 8.8773
Epoch 832/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1659 - val_loss: 8.9267
Epoch 833/1000
1052/1052 [==============================] - 0s 81us/step - loss: 7.8393 - val_loss: 9.1503
Epoch 834/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.6110 - val_loss: 8.3999
Epoch 835/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 164us/step - loss: 7.5880 - val_loss: 9.3170
Epoch 903/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.3582 - val_loss: 8.7570
Epoch 904/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.2436 - val_loss: 8.5242
Epoch 905/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.5533 - val_loss: 8.2357
Epoch 906/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.3598 - val_loss: 8.3694
Epoch 907/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.3317 - val_loss: 8.3237
Epoch 908/1000
1052/1052 [==============================] - 0s 96us/step - loss: 7.4715 - val_loss: 8.1458
Epoch 909/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.1962 - val_loss: 8.5664
Epoch 910/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.4355 - val_loss: 8.3633
Epoch 911/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 88us/step - loss: 7.1152 - val_loss: 8.1664
Epoch 979/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.1752 - val_loss: 8.1241
Epoch 980/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.1141 - val_loss: 9.0826
Epoch 981/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.0540 - val_loss: 9.6236
Epoch 982/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.3812 - val_loss: 8.2963
Epoch 983/1000
1052/1052 [==============================] - 0s 95us/step - loss: 7.2144 - val_loss: 8.3868
Epoch 984/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.1700 - val_loss: 8.0391
Epoch 985/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.9643 - val_loss: 8.4616
Epoch 986/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.9314 - val_loss: 8.0146
Epoch 987/1000
1052/1052 [==========================

[array([[-3.9251552 ,  3.884789  ,  0.88322246,  2.4796124 , -0.5934038 ,
          4.207986  , -0.27696043, -0.28418627, -0.25000414,  0.28551295],
        [-0.36719373, -0.19147044, -0.09302025, -0.12349726, -1.121923  ,
         -0.08937724, -0.16827732, -0.17022455, -0.1597288 ,  1.4026983 ],
        [-0.50859314, -0.09218717,  0.28319323, -0.1698862 , -1.825502  ,
          1.1822263 , -0.35445282, -0.36069253, -0.3296788 ,  1.6317997 ],
        [ 0.29454562, -0.08966386, -0.07818668,  0.20152716, -0.61068124,
         -0.19608739,  0.47655243,  0.4922266 ,  0.41713476, -0.28005102],
        [-3.8370223 ,  0.3724117 ,  0.8380013 ,  0.4623429 , -0.7293927 ,
          0.23067601, -0.27980447, -0.28699806, -0.2540315 , -0.65354174]],
       dtype=float32),
 array([-5.432587  ,  1.9754231 , -0.41158035,  3.2360444 ,  5.588499  ,
         5.098453  ,  2.6730318 ,  2.656358  ,  2.7679121 ,  1.0600914 ],
       dtype=float32),
 array([[-0.3393439 ,  0.29086336, -1.4540825 , -0.98546046, 

In [33]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure4_4th.h5')

In [34]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_5(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 261us/step - l

1052/1052 [==============================] - 0s 90us/step - loss: 14.6765 - val_loss: 17.4997
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.5646 - val_loss: 17.1268
Epoch 69/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.7150 - val_loss: 17.8093
Epoch 70/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.7578 - val_loss: 17.1609
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.4673 - val_loss: 17.3183
Epoch 72/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.3204 - val_loss: 17.1021
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.4359 - val_loss: 18.2927
Epoch 74/1000
1052/1052 [==============================] - 0s 84us/step - loss: 14.4323 - val_loss: 17.7781
Epoch 75/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.6820 - val_loss: 18.6292
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 88us/step - loss: 10.4083 - val_loss: 13.2149
Epoch 143/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.7562 - val_loss: 12.4002
Epoch 144/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.5113 - val_loss: 13.3197
Epoch 145/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.9684 - val_loss: 14.3751
Epoch 146/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.8651 - val_loss: 12.3391
Epoch 147/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.8263 - val_loss: 12.6050
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.1308 - val_loss: 12.9746
Epoch 149/1000
1052/1052 [==============================] - 0s 84us/step - loss: 10.4752 - val_loss: 13.3954
Epoch 150/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.4442 - val_loss: 12.7787
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 87us/step - loss: 7.0128 - val_loss: 7.9258
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.3763 - val_loss: 7.7777
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.0415 - val_loss: 8.4868
Epoch 221/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.1190 - val_loss: 7.8084
Epoch 222/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.1612 - val_loss: 8.1938
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.8969 - val_loss: 7.6031
Epoch 224/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.8842 - val_loss: 7.9029
Epoch 225/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.2790 - val_loss: 8.6460
Epoch 226/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.2284 - val_loss: 8.4486
Epoch 227/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 83us/step - loss: 6.9879 - val_loss: 8.2530
Epoch 295/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.8406 - val_loss: 7.7823
Epoch 296/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.4206 - val_loss: 6.9004
Epoch 297/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.6300 - val_loss: 7.8603
Epoch 298/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.9752 - val_loss: 8.9583
Epoch 299/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.6680 - val_loss: 7.1145
Epoch 300/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.6456 - val_loss: 7.7971
Epoch 301/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.5879 - val_loss: 8.1470
Epoch 302/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.6828 - val_loss: 7.3947
Epoch 303/1000
1052/1052 [==========================

...

1052/1052 [==============================] - 0s 89us/step - loss: 6.7621 - val_loss: 6.5836
Epoch 751/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.7169 - val_loss: 5.5566
Epoch 752/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.4148 - val_loss: 6.1020
Epoch 753/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.6297 - val_loss: 6.2499
Epoch 754/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.5292 - val_loss: 5.7107
Epoch 755/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.5140 - val_loss: 5.3735
Epoch 756/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.9415 - val_loss: 6.0973
Epoch 757/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.4891 - val_loss: 5.7571
Epoch 758/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.7357 - val_loss: 5.7425
Epoch 759/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 88us/step - loss: 5.7859 - val_loss: 5.6140
Epoch 827/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.5006 - val_loss: 5.7151
Epoch 828/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.3442 - val_loss: 6.0456
Epoch 829/1000
1052/1052 [==============================] - 0s 78us/step - loss: 5.3493 - val_loss: 5.6390
Epoch 830/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.4390 - val_loss: 6.2320
Epoch 831/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6865 - val_loss: 5.9355
Epoch 832/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.7933 - val_loss: 6.8134
Epoch 833/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.8161 - val_loss: 5.5316
Epoch 834/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.9052 - val_loss: 6.5604
Epoch 835/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 5.4515 - val_loss: 5.1469
Epoch 903/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.0916 - val_loss: 9.0703
Epoch 904/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.9416 - val_loss: 5.5557
Epoch 905/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.5921 - val_loss: 8.2294
Epoch 906/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.4832 - val_loss: 5.4020
Epoch 907/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.5261 - val_loss: 5.3874
Epoch 908/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.5563 - val_loss: 6.2923
Epoch 909/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.3843 - val_loss: 5.4476
Epoch 910/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.5273 - val_loss: 5.0579
Epoch 911/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 5.3338 - val_loss: 5.3611
Epoch 979/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.4318 - val_loss: 5.6135
Epoch 980/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.3466 - val_loss: 7.0420
Epoch 981/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6480 - val_loss: 5.1613
Epoch 982/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.2483 - val_loss: 5.3211
Epoch 983/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.2846 - val_loss: 5.3949
Epoch 984/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.5894 - val_loss: 6.7025
Epoch 985/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.3927 - val_loss: 5.6683
Epoch 986/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.1098 - val_loss: 6.2871
Epoch 987/1000
1052/1052 [==========================

[array([[-0.5735489 ,  2.965721  , -0.12788029, -4.030838  ,  0.38863236,
          0.53005296, -0.8340661 , -4.231711  , -0.14026332, -4.5569715 ],
        [-0.9878199 , -0.20671615, -0.5639114 ,  0.15339808, -0.00732744,
          0.43364933, -0.850827  , -0.70490915, -0.02871592,  0.30813155],
        [-2.0070765 ,  0.07018906, -1.9282663 ,  0.10152616, -1.1426042 ,
          0.622675  , -0.35386506, -3.002482  ,  0.45451096, -0.7743051 ],
        [-0.36451626, -0.05606051,  0.05223145,  0.03935704, -0.21410394,
         -0.9803505 ,  0.31089288,  0.20114294,  0.6233615 ,  0.17570533],
        [-0.62470955,  0.27751294,  0.8519549 , -2.4516122 ,  1.4873048 ,
          0.46882743, -0.95781964, -0.2426019 , -1.6410679 , -0.253217  ]],
       dtype=float32),
 array([ 5.7879066,  1.7180561, -1.093419 , -4.966212 , -3.1972873,
        -3.5514178, -0.2515836, -4.9571686, -4.7201605, -5.7335043],
       dtype=float32),
 array([[-1.4016353 ,  1.1094826 , -1.4397999 , -1.6250628 ,  1.8238206

In [35]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure5_4th.h5')

In [36]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_6(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_12 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 16        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 256us/step - l

1052/1052 [==============================] - 0s 85us/step - loss: 11.1898 - val_loss: 16.0176
Epoch 68/1000
1052/1052 [==============================] - 0s 85us/step - loss: 11.3662 - val_loss: 15.2408
Epoch 69/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.2059 - val_loss: 13.2690
Epoch 70/1000
1052/1052 [==============================] - 0s 90us/step - loss: 10.7631 - val_loss: 13.3476
Epoch 71/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.8465 - val_loss: 13.1071
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.9374 - val_loss: 12.8270
Epoch 73/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.2135 - val_loss: 16.0735
Epoch 74/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.1794 - val_loss: 12.8483
Epoch 75/1000
1052/1052 [==============================] - 0s 85us/step - loss: 10.5621 - val_loss: 12.6496
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 89us/step - loss: 8.0136 - val_loss: 9.2172
Epoch 144/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.7524 - val_loss: 8.7178
Epoch 145/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.5963 - val_loss: 8.6064
Epoch 146/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.5237 - val_loss: 9.0972
Epoch 147/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.6133 - val_loss: 8.9716
Epoch 148/1000
1052/1052 [==============================] - 0s 95us/step - loss: 9.4202 - val_loss: 10.5295
Epoch 149/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3931 - val_loss: 10.0248
Epoch 150/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.9738 - val_loss: 10.0304
Epoch 151/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1008 - val_loss: 8.6502
Epoch 152/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 85us/step - loss: 9.0281 - val_loss: 10.7462
Epoch 220/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.5684 - val_loss: 12.2906
Epoch 221/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.4513 - val_loss: 11.0042
Epoch 222/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6937 - val_loss: 10.4389
Epoch 223/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.8737 - val_loss: 9.7026
Epoch 224/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.4545 - val_loss: 9.3784
Epoch 225/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.6092 - val_loss: 8.3260
Epoch 226/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.8157 - val_loss: 8.2658
Epoch 227/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1118 - val_loss: 8.5990
Epoch 228/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 82us/step - loss: 7.1795 - val_loss: 8.8613
Epoch 296/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.1791 - val_loss: 12.2739
Epoch 297/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1501 - val_loss: 7.9690
Epoch 298/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.4746 - val_loss: 7.8373
Epoch 299/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.8752 - val_loss: 11.1005
Epoch 300/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.4684 - val_loss: 7.9791
Epoch 301/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.6009 - val_loss: 8.2668
Epoch 302/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.4629 - val_loss: 8.0285
Epoch 303/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.0137 - val_loss: 10.5026
Epoch 304/1000
1052/1052 [=======================

...

1052/1052 [==============================] - 0s 85us/step - loss: 5.7841 - val_loss: 6.1116
Epoch 752/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.1817 - val_loss: 6.3830
Epoch 753/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.0167 - val_loss: 6.1880
Epoch 754/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.4249 - val_loss: 6.0416
Epoch 755/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.5756 - val_loss: 6.4084
Epoch 756/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.4680 - val_loss: 6.4909
Epoch 757/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.7587 - val_loss: 6.1925
Epoch 758/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.7022 - val_loss: 5.8386
Epoch 759/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.3134 - val_loss: 6.3136
Epoch 760/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 85us/step - loss: 5.4625 - val_loss: 6.3317
Epoch 828/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.7192 - val_loss: 5.8895
Epoch 829/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.8727 - val_loss: 6.1063
Epoch 830/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.2766 - val_loss: 6.0297
Epoch 831/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.4270 - val_loss: 6.3621
Epoch 832/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.5143 - val_loss: 6.4283
Epoch 833/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.3205 - val_loss: 7.4873
Epoch 834/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.5001 - val_loss: 6.3951
Epoch 835/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.5837 - val_loss: 6.1317
Epoch 836/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 89us/step - loss: 5.8186 - val_loss: 5.8938
Epoch 904/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.4854 - val_loss: 5.8610
Epoch 905/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.6816 - val_loss: 5.5527
Epoch 906/1000
1052/1052 [==============================] - 0s 94us/step - loss: 5.5047 - val_loss: 6.3429
Epoch 907/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.3692 - val_loss: 5.6984
Epoch 908/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.2897 - val_loss: 5.9540
Epoch 909/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.7534 - val_loss: 5.7520
Epoch 910/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.3318 - val_loss: 5.3187
Epoch 911/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.3390 - val_loss: 5.6154
Epoch 912/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 88us/step - loss: 5.2411 - val_loss: 5.7295
Epoch 980/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.4308 - val_loss: 5.7677
Epoch 981/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.2620 - val_loss: 6.2845
Epoch 982/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.3045 - val_loss: 6.1528
Epoch 983/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.1329 - val_loss: 5.8869
Epoch 984/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.2807 - val_loss: 5.9828
Epoch 985/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.0417 - val_loss: 6.1328
Epoch 986/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.1630 - val_loss: 5.9303
Epoch 987/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.4964 - val_loss: 6.1638
Epoch 988/1000
1052/1052 [==========================

[array([[ 0.9664121 , -0.71439797,  3.457057  , -3.869236  ,  3.1601624 ,
         -0.5665238 , -0.29038548,  1.3507513 ,  3.8137453 , -0.1498905 ],
        [ 0.67139256, -0.32646775, -0.03267581,  0.15531106, -0.20973071,
         -1.3679898 , -0.5755995 ,  1.2874985 ,  0.453728  , -0.08349421],
        [-0.06338986, -0.83258474,  1.6046946 ,  0.09304156,  0.06345822,
         -1.7463905 , -1.3970995 ,  1.772311  ,  1.979807  , -1.1843576 ],
        [-0.2985319 ,  1.1293962 , -0.15580216,  0.00516685, -0.06806385,
          0.01829284,  0.16563173, -0.7386769 , -0.15881743, -0.25415915],
        [ 1.1360056 , -1.141263  , -0.18083592, -2.048034  ,  0.26113388,
         -0.5319344 ,  0.11053264,  2.934145  ,  0.5286733 ,  0.14794125]],
       dtype=float32),
 array([ 0.80044097,  2.9348962 ,  5.4104166 , -4.6731615 ,  1.818548  ,
         5.0096025 , -1.0973121 ,  4.5200706 ,  3.8615472 ,  4.695368  ],
       dtype=float32),
 array([[-0.54518306,  0.1050121 ,  0.6320416 , -0.02741606, 

In [37]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure6_4th.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_1(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_14 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 6         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 799us/step - loss: 5

274/274 [==============================] - 0s 98us/step - loss: 8.3593 - val_loss: 10.0390
Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 8.3811 - val_loss: 10.0354
Epoch 70/1000
274/274 [==============================] - 0s 95us/step - loss: 8.2641 - val_loss: 9.8600
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 8.3064 - val_loss: 9.7282
Epoch 72/1000
274/274 [==============================] - 0s 114us/step - loss: 8.1957 - val_loss: 9.9753
Epoch 73/1000
274/274 [==============================] - 0s 117us/step - loss: 8.1497 - val_loss: 9.8319
Epoch 74/1000
274/274 [==============================] - 0s 100us/step - loss: 8.1458 - val_loss: 9.7401
Epoch 75/1000
274/274 [==============================] - 0s 95us/step - loss: 8.1063 - val_loss: 9.9375
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 8.0672 - val_loss: 9.7403
Epoch 77/1000
274/274 [==============================] - 0s 98us/step - l

274/274 [==============================] - 0s 91us/step - loss: 7.1296 - val_loss: 9.4564
Epoch 147/1000
274/274 [==============================] - 0s 100us/step - loss: 7.3791 - val_loss: 9.5641
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0573 - val_loss: 9.7220
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1622 - val_loss: 9.0841
Epoch 150/1000
274/274 [==============================] - 0s 100us/step - loss: 7.0081 - val_loss: 9.3651
Epoch 151/1000
274/274 [==============================] - 0s 99us/step - loss: 6.9063 - val_loss: 9.5225
Epoch 152/1000
274/274 [==============================] - 0s 96us/step - loss: 7.1337 - val_loss: 9.5243
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3424 - val_loss: 9.5632
Epoch 154/1000
274/274 [==============================] - 0s 95us/step - loss: 6.9469 - val_loss: 9.3175
Epoch 155/1000
274/274 [==============================] - 0s 102us/

Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5470 - val_loss: 8.5305
Epoch 225/1000
274/274 [==============================] - 0s 91us/step - loss: 6.5551 - val_loss: 8.7662
Epoch 226/1000
274/274 [==============================] - 0s 91us/step - loss: 6.5910 - val_loss: 8.5941
Epoch 227/1000
274/274 [==============================] - 0s 101us/step - loss: 6.7838 - val_loss: 8.9780
Epoch 228/1000
274/274 [==============================] - 0s 96us/step - loss: 6.6608 - val_loss: 8.6840
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6862 - val_loss: 8.7233
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6228 - val_loss: 8.5478
Epoch 231/1000
274/274 [==============================] - 0s 97us/step - loss: 6.5313 - val_loss: 8.6643
Epoch 232/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5365 - val_loss: 8.5802
Epoch 233/1000
274/274 [=============================

Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4485 - val_loss: 8.4361
Epoch 303/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4118 - val_loss: 8.3030
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3871 - val_loss: 8.4860
Epoch 305/1000
274/274 [==============================] - 0s 93us/step - loss: 6.3407 - val_loss: 8.4982
Epoch 306/1000
274/274 [==============================] - 0s 100us/step - loss: 6.3131 - val_loss: 8.4524
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5969 - val_loss: 8.3018
Epoch 308/1000
274/274 [==============================] - 0s 96us/step - loss: 6.4396 - val_loss: 8.3584
Epoch 309/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3647 - val_loss: 8.3152
Epoch 310/1000
274/274 [==============================] - 0s 99us/step - loss: 6.3975 - val_loss: 8.3396
Epoch 311/1000
274/274 [============================

...

Epoch 691/1000
274/274 [==============================] - 0s 100us/step - loss: 5.8684 - val_loss: 7.9091
Epoch 692/1000
274/274 [==============================] - 0s 99us/step - loss: 6.0092 - val_loss: 8.0326
Epoch 693/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9331 - val_loss: 7.9985
Epoch 694/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8066 - val_loss: 8.2895
Epoch 695/1000
274/274 [==============================] - 0s 100us/step - loss: 5.8991 - val_loss: 7.9303
Epoch 696/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8777 - val_loss: 8.1737
Epoch 697/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8860 - val_loss: 7.7431
Epoch 698/1000
274/274 [==============================] - 0s 91us/step - loss: 6.0770 - val_loss: 8.0983
Epoch 699/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0422 - val_loss: 7.9779
Epoch 700/1000
274/274 [============================

Epoch 769/1000
274/274 [==============================] - 0s 94us/step - loss: 5.8033 - val_loss: 8.4192
Epoch 770/1000
274/274 [==============================] - 0s 104us/step - loss: 5.7773 - val_loss: 8.4985
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7444 - val_loss: 8.4372
Epoch 772/1000
274/274 [==============================] - 0s 97us/step - loss: 5.8784 - val_loss: 8.3337
Epoch 773/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9133 - val_loss: 8.3535
Epoch 774/1000
274/274 [==============================] - 0s 93us/step - loss: 5.7732 - val_loss: 8.5429
Epoch 775/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7371 - val_loss: 8.3392
Epoch 776/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7468 - val_loss: 8.4229
Epoch 777/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7964 - val_loss: 8.5094
Epoch 778/1000
274/274 [==============================

Epoch 847/1000
274/274 [==============================] - 0s 93us/step - loss: 5.6736 - val_loss: 8.4619
Epoch 848/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7306 - val_loss: 8.5237
Epoch 849/1000
274/274 [==============================] - 0s 99us/step - loss: 5.6517 - val_loss: 9.0393
Epoch 850/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6600 - val_loss: 8.6476
Epoch 851/1000
274/274 [==============================] - 0s 94us/step - loss: 5.6283 - val_loss: 8.8474
Epoch 852/1000
274/274 [==============================] - 0s 100us/step - loss: 5.5946 - val_loss: 8.4139
Epoch 853/1000
274/274 [==============================] - 0s 92us/step - loss: 5.6836 - val_loss: 8.7678
Epoch 854/1000
274/274 [==============================] - 0s 97us/step - loss: 5.6179 - val_loss: 8.8623
Epoch 855/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7976 - val_loss: 9.0387
Epoch 856/1000
274/274 [==============================

Epoch 925/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6056 - val_loss: 8.9691
Epoch 926/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5224 - val_loss: 9.2373
Epoch 927/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6366 - val_loss: 9.1995
Epoch 928/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7723 - val_loss: 9.0028
Epoch 929/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8541 - val_loss: 9.5680
Epoch 930/1000
274/274 [==============================] - 0s 105us/step - loss: 5.5758 - val_loss: 9.0502
Epoch 931/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5988 - val_loss: 9.1018
Epoch 932/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5822 - val_loss: 9.1624
Epoch 933/1000
274/274 [==============================] - 0s 100us/step - loss: 5.5369 - val_loss: 9.2814
Epoch 934/1000
274/274 [===========================

[array([[-0.10562204,  2.8462915 ,  0.06768758, -0.48668215, -0.38481486],
        [-0.21877885,  2.0239153 , -0.18069106, -1.9512062 ,  0.14269303],
        [ 0.03960619, -2.2883415 , -0.0959382 , -0.16708072, -1.1829926 ],
        [-0.3483899 ,  0.32392612,  0.3502195 ,  1.9845966 , -0.7364791 ],
        [ 0.11636996, -0.40747002, -0.5260901 , -1.0818738 , -0.18904634],
        [ 0.02194943,  2.488261  ,  0.4300397 , -1.1068547 , -0.08296724],
        [-0.6891439 , -0.66830826,  1.1318254 , -1.257284  ,  0.70168424]],
       dtype=float32),
 array([-0.17728676,  2.583921  ,  1.4803219 ,  1.9890238 , -0.82225513],
       dtype=float32),
 array([[ 0.4354432 ,  0.9998085 , -0.9778736 ,  0.8812299 , -0.83315015],
        [ 0.02905946, -0.4391056 , -0.8196396 , -0.25135794, -0.6697966 ],
        [ 1.0777968 ,  0.6602865 , -0.3570659 , -0.19244252, -1.3653127 ],
        [-0.31723157, -0.92571056,  0.65161484, -0.1550049 ,  0.44247785],
        [-0.14506157,  0.80129147, -0.47198144,  0.574

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure1_4th.h5')

In [40]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_2(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 972us/step - loss:

274/274 [==============================] - 0s 91us/step - loss: 7.1868 - val_loss: 11.1097
Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1026 - val_loss: 11.4111
Epoch 70/1000
274/274 [==============================] - 0s 101us/step - loss: 7.2778 - val_loss: 11.4580
Epoch 71/1000
274/274 [==============================] - 0s 96us/step - loss: 7.2002 - val_loss: 11.2563
Epoch 72/1000
274/274 [==============================] - 0s 95us/step - loss: 6.9542 - val_loss: 10.6771
Epoch 73/1000
274/274 [==============================] - 0s 100us/step - loss: 6.8473 - val_loss: 10.8607
Epoch 74/1000
274/274 [==============================] - 0s 99us/step - loss: 6.8754 - val_loss: 11.1219
Epoch 75/1000
274/274 [==============================] - 0s 100us/step - loss: 7.0337 - val_loss: 10.8417
Epoch 76/1000
274/274 [==============================] - 0s 100us/step - loss: 7.1379 - val_loss: 10.9070
Epoch 77/1000
274/274 [==============================] - 0s 95us/

274/274 [==============================] - 0s 95us/step - loss: 6.5858 - val_loss: 10.0775
Epoch 146/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3887 - val_loss: 9.9220
Epoch 147/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4115 - val_loss: 9.8632
Epoch 148/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3935 - val_loss: 10.3176
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3435 - val_loss: 10.1075
Epoch 150/1000
274/274 [==============================] - 0s 101us/step - loss: 6.5235 - val_loss: 9.9206
Epoch 151/1000
274/274 [==============================] - 0s 94us/step - loss: 6.3529 - val_loss: 9.9616
Epoch 152/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3363 - val_loss: 10.0511
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5382 - val_loss: 10.3379
Epoch 154/1000
274/274 [==============================] - 0s 9

Epoch 223/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2175 - val_loss: 9.8832
Epoch 224/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3357 - val_loss: 9.8756
Epoch 225/1000
274/274 [==============================] - 0s 99us/step - loss: 6.3047 - val_loss: 9.8609
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2641 - val_loss: 9.8304
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2375 - val_loss: 9.6648
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2522 - val_loss: 9.6102
Epoch 229/1000
274/274 [==============================] - 0s 101us/step - loss: 6.1921 - val_loss: 9.6997
Epoch 230/1000
274/274 [==============================] - 0s 99us/step - loss: 6.2057 - val_loss: 10.0311
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3452 - val_loss: 9.7252
Epoch 232/1000
274/274 [============================

274/274 [==============================] - 0s 98us/step - loss: 6.0588 - val_loss: 9.6562
Epoch 301/1000
274/274 [==============================] - 0s 94us/step - loss: 6.1615 - val_loss: 9.7839
Epoch 302/1000
274/274 [==============================] - 0s 96us/step - loss: 6.1308 - val_loss: 9.5383
Epoch 303/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3741 - val_loss: 9.9843
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3879 - val_loss: 9.6381
Epoch 305/1000
274/274 [==============================] - 0s 131us/step - loss: 6.1511 - val_loss: 9.5809
Epoch 306/1000
274/274 [==============================] - 0s 100us/step - loss: 6.1346 - val_loss: 9.9919
Epoch 307/1000
274/274 [==============================] - 0s 96us/step - loss: 6.4022 - val_loss: 9.7311
Epoch 308/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2065 - val_loss: 9.7994
Epoch 309/1000
274/274 [==============================] - 0s 98us/s

...

274/274 [==============================] - 0s 97us/step - loss: 5.3093 - val_loss: 9.3365
Epoch 765/1000
274/274 [==============================] - 0s 97us/step - loss: 5.2962 - val_loss: 9.4069
Epoch 766/1000
274/274 [==============================] - 0s 99us/step - loss: 5.3416 - val_loss: 9.4694
Epoch 767/1000
274/274 [==============================] - 0s 96us/step - loss: 5.4432 - val_loss: 9.3867
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4266 - val_loss: 9.4315
Epoch 769/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3547 - val_loss: 9.4568
Epoch 770/1000
274/274 [==============================] - 0s 94us/step - loss: 5.3663 - val_loss: 9.4420
Epoch 771/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3155 - val_loss: 9.4865
Epoch 772/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2577 - val_loss: 9.4057
Epoch 773/1000
274/274 [==============================] - 0s 102us/st

Epoch 842/1000
274/274 [==============================] - 0s 111us/step - loss: 5.3547 - val_loss: 9.4493
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3798 - val_loss: 9.5177
Epoch 844/1000
274/274 [==============================] - 0s 116us/step - loss: 5.2350 - val_loss: 9.5164
Epoch 845/1000
274/274 [==============================] - 0s 101us/step - loss: 5.2271 - val_loss: 9.6509
Epoch 846/1000
274/274 [==============================] - 0s 113us/step - loss: 5.3234 - val_loss: 9.4770
Epoch 847/1000
274/274 [==============================] - 0s 107us/step - loss: 5.3486 - val_loss: 9.4702
Epoch 848/1000
274/274 [==============================] - 0s 113us/step - loss: 5.2564 - val_loss: 9.4882
Epoch 849/1000
274/274 [==============================] - 0s 120us/step - loss: 5.4647 - val_loss: 9.4553
Epoch 850/1000
274/274 [==============================] - 0s 116us/step - loss: 5.5771 - val_loss: 9.3801
Epoch 851/1000
274/274 [=======================

274/274 [==============================] - 0s 113us/step - loss: 5.4783 - val_loss: 9.4498
Epoch 920/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4824 - val_loss: 9.5345
Epoch 921/1000
274/274 [==============================] - 0s 104us/step - loss: 5.6740 - val_loss: 9.4325
Epoch 922/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4046 - val_loss: 9.4583
Epoch 923/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3298 - val_loss: 9.4977
Epoch 924/1000
274/274 [==============================] - 0s 105us/step - loss: 5.1916 - val_loss: 9.3364
Epoch 925/1000
274/274 [==============================] - 0s 107us/step - loss: 5.4055 - val_loss: 9.4774
Epoch 926/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3617 - val_loss: 9.5912
Epoch 927/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4920 - val_loss: 9.6128
Epoch 928/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 113us/step - loss: 5.1974 - val_loss: 9.4533
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3337 - val_loss: 9.5281
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2879 - val_loss: 9.5611
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2315 - val_loss: 9.4299
Epoch 1000/1000
274/274 [==============================] - 0s 96us/step - loss: 5.4139 - val_loss: 9.2819
7.108653205936238


[array([[-1.7350274 , -0.8693994 ,  0.92569983, -1.3920354 , -0.32666373],
        [ 2.2008724 ,  2.5540974 , -0.12454545,  0.34559613, -1.1580389 ],
        [-1.887371  , -2.2258608 , -0.24466757,  0.766277  ,  3.9484634 ],
        [-1.8819337 , -2.105869  , -1.2389772 ,  0.44018263,  1.918372  ],
        [-0.22819744,  1.1949954 ,  0.310088  ,  0.941166  ,  0.00507916],
        [ 0.9311101 ,  0.58201647,  0.8559451 , -0.27702054, -1.7395903 ],
        [ 2.0587733 , -0.3602795 , -0.00624899, -2.7764947 , -3.523835  ]],
       dtype=float32),
 array([-0.7352176 ,  0.7305927 , -1.2269505 , -2.1953864 ,  0.73780715],
       dtype=float32),
 array([[-0.3038303 , -0.49257088, -0.76411146, -0.81931883, -0.8029529 ,
         -0.63740236,  0.5434296 ,  0.80913055,  0.5881579 ,  0.91990393],
        [-0.37434456,  0.21276462,  0.37808648, -0.3640574 , -0.6874039 ,
         -0.5364084 , -0.3435534 ,  0.667215  , -0.28354684,  0.5585642 ],
        [-0.60731965, -0.31199494, -0.45507097, -0.95898

In [41]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure2_4th.h5')

In [42]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_3(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_18 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 16        
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 975us/step - loss:

274/274 [==============================] - 0s 101us/step - loss: 8.4689 - val_loss: 9.6043
Epoch 69/1000
274/274 [==============================] - 0s 96us/step - loss: 8.4467 - val_loss: 9.2779
Epoch 70/1000
274/274 [==============================] - 0s 109us/step - loss: 8.2216 - val_loss: 9.2966
Epoch 71/1000
274/274 [==============================] - 0s 116us/step - loss: 8.1003 - val_loss: 8.8309
Epoch 72/1000
274/274 [==============================] - 0s 113us/step - loss: 7.8168 - val_loss: 8.9561
Epoch 73/1000
274/274 [==============================] - 0s 120us/step - loss: 7.5156 - val_loss: 8.5741
Epoch 74/1000
274/274 [==============================] - 0s 106us/step - loss: 7.6966 - val_loss: 8.2728
Epoch 75/1000
274/274 [==============================] - 0s 116us/step - loss: 7.6774 - val_loss: 8.2708
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 7.6060 - val_loss: 8.4678
Epoch 77/1000
274/274 [==============================] - 0s 102us/step

Epoch 146/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1157 - val_loss: 8.6037
Epoch 147/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9909 - val_loss: 8.4709
Epoch 148/1000
274/274 [==============================] - 0s 105us/step - loss: 7.0222 - val_loss: 8.5409
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9586 - val_loss: 8.6266
Epoch 150/1000
274/274 [==============================] - 0s 104us/step - loss: 7.0536 - val_loss: 8.4451
Epoch 151/1000
274/274 [==============================] - 0s 92us/step - loss: 6.9981 - val_loss: 8.4790
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9580 - val_loss: 8.3536
Epoch 153/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1358 - val_loss: 9.1408
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1556 - val_loss: 8.6246
Epoch 155/1000
274/274 [=======================

274/274 [==============================] - 0s 97us/step - loss: 6.8588 - val_loss: 9.5371
Epoch 224/1000
274/274 [==============================] - 0s 100us/step - loss: 6.9512 - val_loss: 9.3684
Epoch 225/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6528 - val_loss: 9.3135
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6051 - val_loss: 9.5144
Epoch 227/1000
274/274 [==============================] - 0s 91us/step - loss: 6.7257 - val_loss: 9.0107
Epoch 228/1000
274/274 [==============================] - 0s 97us/step - loss: 6.6502 - val_loss: 9.1345
Epoch 229/1000
274/274 [==============================] - 0s 93us/step - loss: 6.5986 - val_loss: 8.9302
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5455 - val_loss: 9.1772
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5746 - val_loss: 9.3350
Epoch 232/1000
274/274 [==============================] - 0s 95us/ste

274/274 [==============================] - 0s 95us/step - loss: 6.7252 - val_loss: 9.3898
Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5092 - val_loss: 9.2662
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6482 - val_loss: 9.3011
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4904 - val_loss: 9.3084
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4912 - val_loss: 9.5832
Epoch 305/1000
274/274 [==============================] - 0s 100us/step - loss: 6.6955 - val_loss: 9.4231
Epoch 306/1000
274/274 [==============================] - 0s 96us/step - loss: 6.5753 - val_loss: 9.3702
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4566 - val_loss: 9.3648
Epoch 308/1000
274/274 [==============================] - 0s 116us/step - loss: 6.6721 - val_loss: 9.5947
Epoch 309/1000
274/274 [==============================] - 0s 115us

...

274/274 [==============================] - 0s 98us/step - loss: 6.3040 - val_loss: 8.2366
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2793 - val_loss: 8.0616
Epoch 768/1000
274/274 [==============================] - 0s 99us/step - loss: 6.4029 - val_loss: 8.4920
Epoch 769/1000
274/274 [==============================] - 0s 91us/step - loss: 6.5423 - val_loss: 7.8756
Epoch 770/1000
274/274 [==============================] - 0s 97us/step - loss: 6.4759 - val_loss: 8.2562
Epoch 771/1000
274/274 [==============================] - 0s 93us/step - loss: 6.6452 - val_loss: 8.8481
Epoch 772/1000
274/274 [==============================] - 0s 87us/step - loss: 6.4353 - val_loss: 8.0599
Epoch 773/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3108 - val_loss: 8.2518
Epoch 774/1000
274/274 [==============================] - 0s 94us/step - loss: 6.8991 - val_loss: 8.4306
Epoch 775/1000
274/274 [==============================] - 0s 98us/ste

Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3617 - val_loss: 8.1594
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3965 - val_loss: 8.0323
Epoch 846/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2847 - val_loss: 8.0415
Epoch 847/1000
274/274 [==============================] - 0s 91us/step - loss: 6.2777 - val_loss: 8.1078
Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2526 - val_loss: 8.1283
Epoch 849/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3275 - val_loss: 8.0066
Epoch 850/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3828 - val_loss: 8.4928
Epoch 851/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3399 - val_loss: 8.0423
Epoch 852/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3225 - val_loss: 8.1765
Epoch 853/1000
274/274 [==============================

Epoch 922/1000
274/274 [==============================] - 0s 101us/step - loss: 6.3382 - val_loss: 8.2212
Epoch 923/1000
274/274 [==============================] - 0s 93us/step - loss: 6.3516 - val_loss: 8.0036
Epoch 924/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3796 - val_loss: 8.6253
Epoch 925/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6311 - val_loss: 7.8749
Epoch 926/1000
274/274 [==============================] - 0s 91us/step - loss: 6.2679 - val_loss: 8.4466
Epoch 927/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4134 - val_loss: 8.0700
Epoch 928/1000
274/274 [==============================] - 0s 96us/step - loss: 6.3779 - val_loss: 8.4009
Epoch 929/1000
274/274 [==============================] - 0s 96us/step - loss: 6.4677 - val_loss: 8.5535
Epoch 930/1000
274/274 [==============================] - 0s 99us/step - loss: 6.7375 - val_loss: 8.1301
Epoch 931/1000
274/274 [==============================

Epoch 1000/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3252 - val_loss: 8.0212
6.623743453268277


[array([[ 1.1519048 ,  1.9469985 ,  0.77748704, -1.8380169 ,  0.05035699],
        [ 3.1053884 , -1.3874804 , -0.29227594, -2.2102985 ,  0.80721176],
        [ 0.9040128 ,  0.57624876,  0.42768708,  1.273199  ,  0.35806745],
        [-1.3171207 , -1.370385  ,  0.29683948,  0.435508  ,  0.18232079],
        [-0.34628305, -0.09797841, -0.43961844, -0.00622515,  0.21127266],
        [ 0.83773345, -1.1131687 , -0.19748867, -1.5479099 ,  0.04522674],
        [ 0.166191  ,  0.6960804 ,  0.53037554,  0.01583136, -0.40520555]],
       dtype=float32),
 array([-1.714827  , -3.360453  , -0.51732796, -1.9062803 ,  0.22170988],
       dtype=float32),
 array([[-0.45725298, -0.4331758 , -0.87771916, -0.7171167 , -0.6989963 ,
         -0.02871859, -0.36303493, -0.17308082,  0.06540415, -0.47010872,
         -0.9601795 , -0.00834028,  0.01108593,  0.7847459 ,  0.0597272 ],
        [-0.5235261 , -0.33539498, -0.10342056, -0.5632162 , -0.6217276 ,
         -0.1726563 ,  0.13050693, -0.3157251 ,  0.353823

In [43]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure3_4th.h5')

In [44]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_4(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_19 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_20 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 951us/step - loss

274/274 [==============================] - 0s 97us/step - loss: 6.8895 - val_loss: 9.7422
Epoch 69/1000
274/274 [==============================] - 0s 99us/step - loss: 6.8869 - val_loss: 9.5671
Epoch 70/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8456 - val_loss: 9.8965
Epoch 71/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7809 - val_loss: 9.8021
Epoch 72/1000
274/274 [==============================] - 0s 116us/step - loss: 7.2599 - val_loss: 9.1589
Epoch 73/1000
274/274 [==============================] - 0s 113us/step - loss: 6.6546 - val_loss: 9.5621
Epoch 74/1000
274/274 [==============================] - 0s 97us/step - loss: 6.8843 - val_loss: 9.0955
Epoch 75/1000
274/274 [==============================] - 0s 95us/step - loss: 7.0345 - val_loss: 9.1296
Epoch 76/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6923 - val_loss: 8.8619
Epoch 77/1000
274/274 [==============================] - 0s 95us/step - loss

274/274 [==============================] - 0s 102us/step - loss: 5.2321 - val_loss: 9.0671
Epoch 147/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3918 - val_loss: 8.7617
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5135 - val_loss: 8.9917
Epoch 149/1000
274/274 [==============================] - 0s 96us/step - loss: 5.3617 - val_loss: 9.1182
Epoch 150/1000
274/274 [==============================] - 0s 100us/step - loss: 5.3986 - val_loss: 8.9629
Epoch 151/1000
274/274 [==============================] - 0s 94us/step - loss: 5.3621 - val_loss: 8.8400
Epoch 152/1000
274/274 [==============================] - 0s 93us/step - loss: 5.0916 - val_loss: 9.0955
Epoch 153/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6988 - val_loss: 9.3879
Epoch 154/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4487 - val_loss: 9.6753
Epoch 155/1000
274/274 [==============================] - 0s 98us/

Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1244 - val_loss: 9.2635
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0326 - val_loss: 9.7648
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0231 - val_loss: 9.5821
Epoch 227/1000
274/274 [==============================] - 0s 87us/step - loss: 4.9599 - val_loss: 10.5638
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2746 - val_loss: 9.1502
Epoch 229/1000
274/274 [==============================] - 0s 101us/step - loss: 4.9046 - val_loss: 9.5121
Epoch 230/1000
274/274 [==============================] - 0s 97us/step - loss: 4.8507 - val_loss: 9.4076
Epoch 231/1000
274/274 [==============================] - 0s 95us/step - loss: 4.9777 - val_loss: 9.5430
Epoch 232/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9539 - val_loss: 9.4797
Epoch 233/1000
274/274 [=============================

274/274 [==============================] - 0s 116us/step - loss: 4.8916 - val_loss: 9.7342
Epoch 302/1000
274/274 [==============================] - 0s 120us/step - loss: 4.9480 - val_loss: 9.3759
Epoch 303/1000
274/274 [==============================] - 0s 116us/step - loss: 4.9368 - val_loss: 10.1469
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 4.9560 - val_loss: 9.3335
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2206 - val_loss: 9.7968
Epoch 306/1000
274/274 [==============================] - 0s 109us/step - loss: 4.8569 - val_loss: 9.1291
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 4.7607 - val_loss: 9.5131
Epoch 308/1000
274/274 [==============================] - 0s 97us/step - loss: 4.8003 - val_loss: 9.9401
Epoch 309/1000
274/274 [==============================] - 0s 107us/step - loss: 4.7533 - val_loss: 9.2640
Epoch 310/1000
274/274 [==============================] - 0s 11

...

274/274 [==============================] - 0s 98us/step - loss: 4.0700 - val_loss: 10.3290
Epoch 764/1000
274/274 [==============================] - 0s 105us/step - loss: 4.0006 - val_loss: 9.5257
Epoch 765/1000
274/274 [==============================] - 0s 113us/step - loss: 3.9202 - val_loss: 10.1011
Epoch 766/1000
274/274 [==============================] - 0s 104us/step - loss: 3.9978 - val_loss: 9.9854
Epoch 767/1000
274/274 [==============================] - 0s 94us/step - loss: 3.9195 - val_loss: 9.5125
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9107 - val_loss: 10.3425
Epoch 769/1000
274/274 [==============================] - 0s 110us/step - loss: 4.0812 - val_loss: 9.5420
Epoch 770/1000
274/274 [==============================] - 0s 106us/step - loss: 3.8478 - val_loss: 10.8176
Epoch 771/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9734 - val_loss: 9.2237
Epoch 772/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 109us/step - loss: 3.9263 - val_loss: 10.0166
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8244 - val_loss: 9.7422
Epoch 842/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9005 - val_loss: 9.7356
Epoch 843/1000
274/274 [==============================] - 0s 105us/step - loss: 3.8436 - val_loss: 10.0716
Epoch 844/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8390 - val_loss: 9.9334
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9146 - val_loss: 9.5100
Epoch 846/1000
274/274 [==============================] - 0s 100us/step - loss: 4.0883 - val_loss: 11.2128
Epoch 847/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1647 - val_loss: 9.4809
Epoch 848/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9739 - val_loss: 9.9317
Epoch 849/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 95us/step - loss: 3.8597 - val_loss: 10.1858
Epoch 918/1000
274/274 [==============================] - 0s 107us/step - loss: 3.7987 - val_loss: 10.1094
Epoch 919/1000
274/274 [==============================] - 0s 104us/step - loss: 3.8607 - val_loss: 9.6712
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9369 - val_loss: 9.9918
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8255 - val_loss: 11.2392
Epoch 922/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9566 - val_loss: 9.8850
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8457 - val_loss: 9.8409
Epoch 924/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9724 - val_loss: 10.0541
Epoch 925/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9468 - val_loss: 9.6314
Epoch 926/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 102us/step - loss: 3.6496 - val_loss: 10.1971
Epoch 995/1000
274/274 [==============================] - 0s 106us/step - loss: 3.7108 - val_loss: 10.6330
Epoch 996/1000
274/274 [==============================] - 0s 109us/step - loss: 3.7394 - val_loss: 10.0274
Epoch 997/1000
274/274 [==============================] - 0s 108us/step - loss: 3.7341 - val_loss: 10.2301
Epoch 998/1000
274/274 [==============================] - 0s 103us/step - loss: 3.7724 - val_loss: 10.1504
Epoch 999/1000
274/274 [==============================] - 0s 96us/step - loss: 3.8608 - val_loss: 10.2773
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8163 - val_loss: 10.3105
9.321624642711575


[array([[-0.01587684,  3.3961208 ,  0.9976807 , -1.8066075 ,  0.76832217,
         -1.4006512 , -0.9553555 ,  1.5285815 ,  0.15097545, -1.5270396 ],
        [ 0.54927874, -1.6584923 , -0.49496475,  0.56334066, -0.6029632 ,
          0.5443326 , -1.3201429 , -3.9735973 , -0.21326391,  0.8425982 ],
        [-0.28603598,  2.3327014 , -0.8422549 , -0.5274472 , -0.27047464,
          0.5659955 ,  0.05911589, -2.7950149 , -0.93200237,  0.08850259],
        [-1.2293054 ,  0.91389924,  0.8314568 ,  1.340507  ,  1.1333774 ,
         -0.6399632 ,  1.3495536 ,  0.89416045, -0.05827907, -0.24505417],
        [ 0.8172982 ,  0.05027466, -0.775708  , -0.11928917, -0.36958307,
          0.16335952, -0.09156059,  0.48534945, -1.2946714 ,  0.57557124],
        [-0.6450549 , -0.7623523 ,  0.60316765, -0.10105048, -1.77633   ,
          0.4218894 ,  2.4833734 ,  1.112772  , -0.7942972 , -0.68007535],
        [-1.1292084 , -1.2164863 , -0.11383384,  0.5946333 , -0.01031957,
         -0.20427616,  2.2784922

In [45]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure4_4th.h5')

In [46]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_5(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_21 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

274/274 [==============================] - 0s 118us/step - loss: 5.8147 - val_loss: 9.2992
Epoch 69/1000
274/274 [==============================] - 0s 106us/step - loss: 5.9808 - val_loss: 9.2398
Epoch 70/1000
274/274 [==============================] - 0s 108us/step - loss: 5.8437 - val_loss: 9.3566
Epoch 71/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7687 - val_loss: 9.1267
Epoch 72/1000
274/274 [==============================] - 0s 107us/step - loss: 5.7291 - val_loss: 9.4236
Epoch 73/1000
274/274 [==============================] - 0s 110us/step - loss: 5.8106 - val_loss: 8.9994
Epoch 74/1000
274/274 [==============================] - 0s 106us/step - loss: 5.8137 - val_loss: 9.6131
Epoch 75/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1210 - val_loss: 9.1065
Epoch 76/1000
274/274 [==============================] - 0s 116us/step - loss: 6.0145 - val_loss: 9.4919
Epoch 77/1000
274/274 [==============================] - 0s 112us/ste

274/274 [==============================] - 0s 102us/step - loss: 5.3073 - val_loss: 8.4900
Epoch 146/1000
274/274 [==============================] - 0s 103us/step - loss: 5.0634 - val_loss: 8.4062
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0754 - val_loss: 8.9083
Epoch 148/1000
274/274 [==============================] - 0s 92us/step - loss: 5.1349 - val_loss: 8.6007
Epoch 149/1000
274/274 [==============================] - 0s 94us/step - loss: 5.2109 - val_loss: 9.0776
Epoch 150/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2037 - val_loss: 8.8152
Epoch 151/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1358 - val_loss: 8.6097
Epoch 152/1000
274/274 [==============================] - 0s 100us/step - loss: 5.1873 - val_loss: 8.6003
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1493 - val_loss: 9.0920
Epoch 154/1000
274/274 [==============================] - 0s 98us/s

Epoch 223/1000
274/274 [==============================] - 0s 95us/step - loss: 4.8543 - val_loss: 8.5253
Epoch 224/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7975 - val_loss: 8.3115
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8260 - val_loss: 8.5696
Epoch 226/1000
274/274 [==============================] - 0s 113us/step - loss: 4.8185 - val_loss: 8.6381
Epoch 227/1000
274/274 [==============================] - 0s 113us/step - loss: 4.7797 - val_loss: 8.9012
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 4.6780 - val_loss: 8.2314
Epoch 229/1000
274/274 [==============================] - 0s 116us/step - loss: 4.9753 - val_loss: 8.6620
Epoch 230/1000
274/274 [==============================] - 0s 113us/step - loss: 4.8620 - val_loss: 8.5398
Epoch 231/1000
274/274 [==============================] - 0s 109us/step - loss: 4.7164 - val_loss: 8.5282
Epoch 232/1000
274/274 [========================

274/274 [==============================] - 0s 98us/step - loss: 4.4590 - val_loss: 8.9107
Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5704 - val_loss: 8.3863
Epoch 302/1000
274/274 [==============================] - 0s 101us/step - loss: 4.6234 - val_loss: 8.2059
Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5939 - val_loss: 8.3730
Epoch 304/1000
274/274 [==============================] - 0s 101us/step - loss: 4.5878 - val_loss: 8.5477
Epoch 305/1000
274/274 [==============================] - 0s 98us/step - loss: 4.6229 - val_loss: 8.9638
Epoch 306/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5238 - val_loss: 8.4984
Epoch 307/1000
274/274 [==============================] - 0s 96us/step - loss: 4.5024 - val_loss: 8.7696
Epoch 308/1000
274/274 [==============================] - 0s 101us/step - loss: 4.5429 - val_loss: 8.9251
Epoch 309/1000
274/274 [==============================] - 0s 95us/s

...

Epoch 762/1000
274/274 [==============================] - 0s 120us/step - loss: 4.1360 - val_loss: 10.9585
Epoch 763/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9649 - val_loss: 10.6265
Epoch 764/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0454 - val_loss: 10.9782
Epoch 765/1000
274/274 [==============================] - 0s 119us/step - loss: 4.0684 - val_loss: 10.7562
Epoch 766/1000
274/274 [==============================] - 0s 109us/step - loss: 4.0358 - val_loss: 11.0661
Epoch 767/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1065 - val_loss: 11.0988
Epoch 768/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2663 - val_loss: 10.7265
Epoch 769/1000
274/274 [==============================] - 0s 131us/step - loss: 4.0301 - val_loss: 10.7514
Epoch 770/1000
274/274 [==============================] - 0s 113us/step - loss: 4.1036 - val_loss: 10.9790
Epoch 771/1000
274/274 [===============

274/274 [==============================] - 0s 106us/step - loss: 4.0765 - val_loss: 11.4587
Epoch 839/1000
274/274 [==============================] - 0s 116us/step - loss: 4.1248 - val_loss: 11.3223
Epoch 840/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1880 - val_loss: 10.9040
Epoch 841/1000
274/274 [==============================] - 0s 103us/step - loss: 4.0739 - val_loss: 12.0142
Epoch 842/1000
274/274 [==============================] - 0s 110us/step - loss: 4.0598 - val_loss: 11.0591
Epoch 843/1000
274/274 [==============================] - 0s 99us/step - loss: 4.1192 - val_loss: 12.3421
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4988 - val_loss: 11.2449
Epoch 845/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2671 - val_loss: 10.8294
Epoch 846/1000
274/274 [==============================] - 0s 113us/step - loss: 4.2244 - val_loss: 12.3807
Epoch 847/1000
274/274 [==============================

274/274 [==============================] - 0s 96us/step - loss: 4.0119 - val_loss: 11.5810
Epoch 916/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9481 - val_loss: 11.9869
Epoch 917/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9457 - val_loss: 11.3802
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0641 - val_loss: 11.5832
Epoch 919/1000
274/274 [==============================] - 0s 93us/step - loss: 3.9774 - val_loss: 11.4650
Epoch 920/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9394 - val_loss: 11.8471
Epoch 921/1000
274/274 [==============================] - 0s 96us/step - loss: 4.0522 - val_loss: 12.1762
Epoch 922/1000
274/274 [==============================] - 0s 89us/step - loss: 4.3450 - val_loss: 11.7618
Epoch 923/1000
274/274 [==============================] - 0s 87us/step - loss: 4.4056 - val_loss: 12.3073
Epoch 924/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 96us/step - loss: 4.6696 - val_loss: 13.1119
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4514 - val_loss: 11.6028
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2213 - val_loss: 11.7268
Epoch 995/1000
274/274 [==============================] - 0s 93us/step - loss: 4.0077 - val_loss: 11.5874
Epoch 996/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9228 - val_loss: 11.7785
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1271 - val_loss: 11.9955
Epoch 998/1000
274/274 [==============================] - 0s 93us/step - loss: 4.1130 - val_loss: 11.7044
Epoch 999/1000
274/274 [==============================] - 0s 90us/step - loss: 3.9191 - val_loss: 11.9441
Epoch 1000/1000
274/274 [==============================] - 0s 101us/step - loss: 3.8885 - val_loss: 11.9091
10.610035621513754


[array([[ 1.1639695 ,  1.283826  , -1.6898532 , -2.0601327 , -1.626264  ,
         -1.3830483 , -0.93661493,  0.41437373, -1.2904012 , -1.0984253 ],
        [-1.2813449 , -0.8490455 , -0.23721956, -0.5582743 ,  0.30940345,
          1.4459466 , -1.4134902 ,  1.3078264 ,  1.2726995 ,  4.1178393 ],
        [-1.7363635 , -0.44613016, -0.04229394,  1.9745954 ,  0.16846931,
         -0.83163345,  0.9209264 ,  1.0743065 ,  0.21251558,  2.4405181 ],
        [ 1.0470659 ,  1.261406  ,  1.3089259 ,  4.2053304 ,  0.14426939,
          0.44580382,  1.6901729 , -1.0119238 , -1.3332785 , -1.1992948 ],
        [ 1.2032675 , -0.65293646, -1.2980282 , -0.14346465,  0.44082323,
          0.71259   , -0.04488189,  0.3742145 ,  0.19019254, -0.55804586],
        [-0.99392384, -1.7542171 ,  1.2657083 ,  0.4681495 ,  0.50954723,
         -1.3582423 ,  0.01497945,  0.4688064 ,  1.7239367 , -0.4600185 ],
        [ 0.5730523 , -0.5396366 ,  2.3695383 , -0.5783853 ,  0.15894355,
         -0.23279822,  0.4629590

In [47]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure5_4th.h5')

In [48]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_6(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_23 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_24 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 16        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

274/274 [==============================] - 0s 91us/step - loss: 6.1084 - val_loss: 8.4643
Epoch 70/1000
274/274 [==============================] - 0s 91us/step - loss: 6.4549 - val_loss: 8.4438
Epoch 71/1000
274/274 [==============================] - 0s 94us/step - loss: 6.4991 - val_loss: 8.1932
Epoch 72/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0135 - val_loss: 9.0321
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3215 - val_loss: 8.8011
Epoch 74/1000
274/274 [==============================] - 0s 99us/step - loss: 6.2612 - val_loss: 7.8989
Epoch 75/1000
274/274 [==============================] - 0s 92us/step - loss: 5.9797 - val_loss: 8.0604
Epoch 76/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8681 - val_loss: 7.9024
Epoch 77/1000
274/274 [==============================] - 0s 88us/step - loss: 6.1052 - val_loss: 8.4838
Epoch 78/1000
274/274 [==============================] - 0s 95us/step - loss: 

274/274 [==============================] - 0s 95us/step - loss: 5.3076 - val_loss: 9.0301
Epoch 148/1000
274/274 [==============================] - 0s 96us/step - loss: 5.3261 - val_loss: 8.4740
Epoch 149/1000
274/274 [==============================] - 0s 94us/step - loss: 5.2774 - val_loss: 8.1230
Epoch 150/1000
274/274 [==============================] - 0s 87us/step - loss: 5.3263 - val_loss: 8.0595
Epoch 151/1000
274/274 [==============================] - 0s 87us/step - loss: 5.9487 - val_loss: 8.9467
Epoch 152/1000
274/274 [==============================] - 0s 87us/step - loss: 5.2352 - val_loss: 8.5087
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3649 - val_loss: 8.7053
Epoch 154/1000
274/274 [==============================] - 0s 96us/step - loss: 5.1606 - val_loss: 8.2323
Epoch 155/1000
274/274 [==============================] - 0s 87us/step - loss: 5.3526 - val_loss: 8.1887
Epoch 156/1000
274/274 [==============================] - 0s 91us/step

274/274 [==============================] - 0s 91us/step - loss: 4.9533 - val_loss: 9.4128
Epoch 226/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4969 - val_loss: 8.8847
Epoch 227/1000
274/274 [==============================] - 0s 91us/step - loss: 5.0008 - val_loss: 8.8489
Epoch 228/1000
274/274 [==============================] - 0s 87us/step - loss: 5.1219 - val_loss: 9.1341
Epoch 229/1000
274/274 [==============================] - 0s 92us/step - loss: 4.7739 - val_loss: 9.2499
Epoch 230/1000
274/274 [==============================] - 0s 92us/step - loss: 4.8671 - val_loss: 8.7672
Epoch 231/1000
274/274 [==============================] - 0s 91us/step - loss: 4.9766 - val_loss: 8.9252
Epoch 232/1000
274/274 [==============================] - 0s 91us/step - loss: 4.8046 - val_loss: 8.9312
Epoch 233/1000
274/274 [==============================] - 0s 91us/step - loss: 4.9320 - val_loss: 9.8623
Epoch 234/1000
274/274 [==============================] - 0s 90us/step

274/274 [==============================] - 0s 101us/step - loss: 4.5462 - val_loss: 8.7549
Epoch 303/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7019 - val_loss: 9.3100
Epoch 304/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7233 - val_loss: 8.6442
Epoch 305/1000
274/274 [==============================] - 0s 108us/step - loss: 4.4909 - val_loss: 8.8078
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7037 - val_loss: 9.2799
Epoch 307/1000
274/274 [==============================] - 0s 109us/step - loss: 4.5130 - val_loss: 8.8287
Epoch 308/1000
274/274 [==============================] - 0s 99us/step - loss: 4.5358 - val_loss: 8.7474
Epoch 309/1000
274/274 [==============================] - 0s 91us/step - loss: 4.6187 - val_loss: 9.4222
Epoch 310/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7417 - val_loss: 8.8898
Epoch 311/1000
274/274 [==============================] - 0s 98u

...

274/274 [==============================] - 0s 97us/step - loss: 3.8110 - val_loss: 8.2081
Epoch 767/1000
274/274 [==============================] - 0s 92us/step - loss: 3.9821 - val_loss: 9.2108
Epoch 768/1000
274/274 [==============================] - 0s 92us/step - loss: 4.4691 - val_loss: 7.9309
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3192 - val_loss: 8.7376
Epoch 770/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1748 - val_loss: 8.3286
Epoch 771/1000
274/274 [==============================] - 0s 106us/step - loss: 4.3454 - val_loss: 10.2837
Epoch 772/1000
274/274 [==============================] - 0s 94us/step - loss: 5.4246 - val_loss: 8.3727
Epoch 773/1000
274/274 [==============================] - 0s 127us/step - loss: 4.2293 - val_loss: 8.1020
Epoch 774/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9988 - val_loss: 7.6522
Epoch 775/1000
274/274 [==============================] - 0s 92us

274/274 [==============================] - 0s 91us/step - loss: 3.9935 - val_loss: 7.9195
Epoch 845/1000
274/274 [==============================] - 0s 91us/step - loss: 3.9723 - val_loss: 10.0763
Epoch 846/1000
274/274 [==============================] - 0s 91us/step - loss: 4.7135 - val_loss: 8.0301
Epoch 847/1000
274/274 [==============================] - 0s 87us/step - loss: 4.3445 - val_loss: 8.2083
Epoch 848/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2806 - val_loss: 8.9216
Epoch 849/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0214 - val_loss: 8.4577
Epoch 850/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9544 - val_loss: 8.3197
Epoch 851/1000
274/274 [==============================] - 0s 96us/step - loss: 3.9744 - val_loss: 8.2019
Epoch 852/1000
274/274 [==============================] - 0s 94us/step - loss: 4.0336 - val_loss: 8.6923
Epoch 853/1000
274/274 [==============================] - 0s 91us/st

274/274 [==============================] - 0s 92us/step - loss: 3.7760 - val_loss: 8.3142
Epoch 922/1000
274/274 [==============================] - 0s 91us/step - loss: 3.8925 - val_loss: 8.4062
Epoch 923/1000
274/274 [==============================] - 0s 89us/step - loss: 3.8197 - val_loss: 9.2279
Epoch 924/1000
274/274 [==============================] - 0s 91us/step - loss: 3.8075 - val_loss: 8.0821
Epoch 925/1000
274/274 [==============================] - 0s 91us/step - loss: 4.2057 - val_loss: 9.4456
Epoch 926/1000
274/274 [==============================] - 0s 87us/step - loss: 4.0052 - val_loss: 8.3800
Epoch 927/1000
274/274 [==============================] - 0s 95us/step - loss: 4.3441 - val_loss: 9.2543
Epoch 928/1000
274/274 [==============================] - 0s 87us/step - loss: 4.6643 - val_loss: 8.9844
Epoch 929/1000
274/274 [==============================] - 0s 91us/step - loss: 3.9892 - val_loss: 9.2712
Epoch 930/1000
274/274 [==============================] - 0s 87us/step

274/274 [==============================] - 0s 91us/step - loss: 3.9615 - val_loss: 8.5161
Epoch 999/1000
274/274 [==============================] - 0s 95us/step - loss: 3.6941 - val_loss: 9.1120
Epoch 1000/1000
274/274 [==============================] - 0s 93us/step - loss: 3.7523 - val_loss: 9.2952
7.553562196634584


[array([[-0.12028391, -1.1374549 ,  1.736599  , -1.7302289 ,  2.4218044 ,
         -1.8547182 , -1.6800588 , -0.6686689 ,  0.28830576,  0.5316882 ],
        [-0.75377953, -1.7412553 , -0.6585252 ,  0.08620577,  5.18228   ,
          0.66336673,  2.0415156 , -1.789034  ,  1.263134  ,  0.05448996],
        [-1.0817478 , -2.6584344 ,  0.10593318,  0.5400321 ,  2.0224674 ,
         -0.9505978 , -0.27094358, -0.23341653, -0.9349026 , -0.66994953],
        [ 1.5208857 , -0.8365863 , -0.19778126,  0.509167  ,  1.7467809 ,
         -0.26585358, -1.4330475 ,  2.1550283 , -0.61239004,  2.5562274 ],
        [-0.05223066,  2.2953107 ,  0.3800391 ,  0.31630427, -0.7647919 ,
         -0.21774533,  1.0795949 , -0.33548716, -0.05881788, -0.97525305],
        [-1.2680159 ,  1.3281231 , -0.31633088, -0.3938801 , -1.2302382 ,
          0.33904576, -0.663743  , -1.531681  ,  1.9748439 , -0.827203  ],
        [-1.6552871 ,  2.4402447 , -0.40009326, -0.5416761 , -1.1442833 ,
          0.648384  ,  0.8719784

In [49]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure6_4th.h5')

In [50]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_1(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_26 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 6         
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0039
Epoch 70/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 71/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0052 - val_loss: 0.0045
Epoch 72/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0055 - val_loss: 0.0042
Epoch 73/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0052 - val_loss: 0.0048
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0041
Epoch 75/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0048 - val_loss: 0.0045
Epoch 76/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0051 - val_loss: 0.0043
Epoch 77/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0053 - val_loss: 0.0064
Epoch 78/1000
143/143 [==============================] - 0s 96us/step 

Epoch 147/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0037 - val_loss: 0.0048
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0047
Epoch 149/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0039 - val_loss: 0.0048
Epoch 150/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0039 - val_loss: 0.0044
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0044
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0050
Epoch 153/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0039 - val_loss: 0.0043
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0041
Epoch 155/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0038 - val_loss: 0.0068
Epoch 156/1000
143/143 [======================

143/143 [==============================] - 0s 101us/step - loss: 0.0030 - val_loss: 0.0043
Epoch 225/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0048
Epoch 226/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0028 - val_loss: 0.0044
Epoch 227/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0028 - val_loss: 0.0041
Epoch 228/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0033 - val_loss: 0.0048
Epoch 229/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0030 - val_loss: 0.0043
Epoch 230/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0029 - val_loss: 0.0042
Epoch 231/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0030 - val_loss: 0.0060
Epoch 232/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0032 - val_loss: 0.0045
Epoch 233/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0023 - val_loss: 0.0041
Epoch 302/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0023 - val_loss: 0.0048
Epoch 303/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0025 - val_loss: 0.0043
Epoch 304/1000
143/143 [==============================] - 0s 99us/step - loss: 0.0023 - val_loss: 0.0046
Epoch 305/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0023 - val_loss: 0.0045
Epoch 306/1000
143/143 [==============================] - 0s 102us/step - loss: 0.0027 - val_loss: 0.0043
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0027 - val_loss: 0.0052
Epoch 308/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0027 - val_loss: 0.0045
Epoch 309/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0030 - val_loss: 0.0044
Epoch 310/1000
143/143 [==============================] - 0s 105

...

143/143 [==============================] - 0s 109us/step - loss: 0.0014 - val_loss: 0.0050
Epoch 764/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0053
Epoch 765/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0049
Epoch 766/1000
143/143 [==============================] - 0s 92us/step - loss: 0.0013 - val_loss: 0.0049
Epoch 767/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 768/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0013 - val_loss: 0.0052
Epoch 769/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0014 - val_loss: 0.0050
Epoch 770/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0013 - val_loss: 0.0050
Epoch 771/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0013 - val_loss: 0.0052
Epoch 772/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0065
Epoch 841/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0016 - val_loss: 0.0051
Epoch 842/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0015 - val_loss: 0.0051
Epoch 843/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0052
Epoch 844/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0014 - val_loss: 0.0055
Epoch 845/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0018 - val_loss: 0.0053
Epoch 846/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0014 - val_loss: 0.0054
Epoch 847/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0012 - val_loss: 0.0047
Epoch 848/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0013 - val_loss: 0.0050
Epoch 849/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0054
Epoch 918/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0015 - val_loss: 0.0051
Epoch 919/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0062
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0051
Epoch 921/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0049
Epoch 922/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0015 - val_loss: 0.0056
Epoch 923/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0012 - val_loss: 0.0052
Epoch 924/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0013 - val_loss: 0.0053
Epoch 925/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0052
Epoch 926/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 133us/step - loss: 0.0023 - val_loss: 0.0056
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0061
Epoch 996/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0022 - val_loss: 0.0056
Epoch 997/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0020 - val_loss: 0.0064
Epoch 998/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0018 - val_loss: 0.0053
Epoch 999/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0015 - val_loss: 0.0052
Epoch 1000/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0013 - val_loss: 0.0052
0.008062931708991528


[array([[ 5.4572242e-01,  8.1395912e-01, -8.1829512e-01, -4.9526799e-01,
         -7.1121371e-01],
        [ 9.8678023e-02, -2.7170020e-01,  5.8043122e-01, -5.6454343e-01,
          3.8592598e-01],
        [ 1.8328462e-02,  1.4726886e-01,  7.2901225e-01, -6.4288139e-01,
          3.0222836e-01],
        [-1.0864085e-02,  4.6984279e-01,  2.7233416e-01, -6.9548005e-01,
          1.0841838e+00],
        [-2.9395930e-03, -1.9265683e-02,  4.0132487e-01, -1.4887788e+00,
         -1.4735620e-02],
        [-8.5325670e-01, -7.3470008e-01, -2.7525559e-01, -4.4424683e-01,
         -8.6709291e-01],
        [ 3.9613715e-01, -4.0190715e-01,  4.5254010e-01, -1.2290791e+00,
         -8.7363046e-01],
        [ 2.9471648e-01,  3.6090896e-01,  2.1694399e-01, -2.6279090e-02,
         -2.6558207e-03],
        [-3.0949569e-01, -7.9460688e-02, -3.4442896e-01, -9.0982372e-01,
         -3.5798791e-01],
        [ 2.0911319e+00,  8.7794930e-01,  3.3261800e-01, -3.4233910e-01,
          6.6050701e-04],
        [ 

In [51]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure1_4th.h5')

In [52]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_2(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 110us/step - loss: 0.0061 - val_loss: 0.0087
Epoch 70/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0069 - val_loss: 0.0052
Epoch 71/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0066 - val_loss: 0.0044
Epoch 72/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0070 - val_loss: 0.0060
Epoch 73/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0057 - val_loss: 0.0052
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0051 - val_loss: 0.0045
Epoch 75/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0063 - val_loss: 0.0062
Epoch 76/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0054 - val_loss: 0.0043
Epoch 77/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0053 - val_loss: 0.0051
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 107us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 147/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 148/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0051
Epoch 149/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0036 - val_loss: 0.0046
Epoch 150/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0051
Epoch 151/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0036 - val_loss: 0.0046
Epoch 152/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0039 - val_loss: 0.0045
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0057
Epoch 154/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0034 - val_loss: 0.0047
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0054
Epoch 224/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0034 - val_loss: 0.0045
Epoch 225/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0032 - val_loss: 0.0047
Epoch 226/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0034 - val_loss: 0.0052
Epoch 227/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0031 - val_loss: 0.0045
Epoch 228/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0029 - val_loss: 0.0044
Epoch 229/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0031 - val_loss: 0.0055
Epoch 230/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0033 - val_loss: 0.0042
Epoch 231/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0031 - val_loss: 0.0044
Epoch 232/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 110us/step - loss: 0.0028 - val_loss: 0.0041
Epoch 301/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0026 - val_loss: 0.0045
Epoch 302/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0026 - val_loss: 0.0049
Epoch 303/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0035 - val_loss: 0.0061
Epoch 304/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0033 - val_loss: 0.0040
Epoch 305/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0030 - val_loss: 0.0039
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0049
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0040
Epoch 308/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 113us/step - loss: 0.0016 - val_loss: 0.0041
Epoch 763/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0016 - val_loss: 0.0036
Epoch 764/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0019 - val_loss: 0.0036
Epoch 765/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0017 - val_loss: 0.0045
Epoch 766/1000
143/143 [==============================] - 0s 102us/step - loss: 0.0018 - val_loss: 0.0036
Epoch 767/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0016 - val_loss: 0.0040
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0036
Epoch 769/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 770/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0038
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0036
Epoch 840/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 841/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 842/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0015 - val_loss: 0.0036
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0037
Epoch 844/1000
143/143 [==============================] - 0s 99us/step - loss: 0.0015 - val_loss: 0.0043
Epoch 845/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0036
Epoch 846/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 847/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0019 - val_loss: 0.0037
Epoch 848/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 917/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0037
Epoch 918/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0037
Epoch 919/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0038
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0038
Epoch 921/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 922/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0016 - val_loss: 0.0041
Epoch 923/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0037
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0036
Epoch 925/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 995/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0017 - val_loss: 0.0039
Epoch 996/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0012 - val_loss: 0.0046
Epoch 997/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0015 - val_loss: 0.0040
Epoch 998/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0016 - val_loss: 0.0042
Epoch 999/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 1000/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0013 - val_loss: 0.0038
0.004593052435666323


[array([[-0.60619414,  0.53706574, -0.29817966,  0.9125664 , -0.17468823],
        [ 0.7411126 , -0.9988812 , -0.44103047, -0.61917883,  0.43020818],
        [ 0.39505866,  0.7254051 ,  0.14011933, -0.48404396,  0.02504235],
        [ 0.06455621, -0.74727225, -0.52423483, -0.05501774,  0.3111043 ],
        [ 0.00625989, -0.05894499, -0.05519401,  0.0470604 ,  0.07775923],
        [-0.06032483, -1.1925793 , -0.6208793 ,  0.5000313 ,  1.0711833 ],
        [ 0.54988116, -0.49205956, -0.39470723, -0.7497958 , -0.19201355],
        [ 0.17021699,  0.60853815,  0.15311064, -0.10946495, -0.4143899 ],
        [ 0.20631644, -0.07015824,  0.21240926,  0.9206695 ,  0.77650505],
        [ 0.6470893 ,  0.83835155, -0.63250273, -0.65021753, -1.926586  ],
        [ 0.9615244 , -0.45782664,  0.3860102 ,  0.2041019 , -0.7043702 ],
        [-1.0853072 , -0.1553475 ,  1.0047957 , -0.9688398 , -0.9482677 ],
        [-0.2315692 , -0.4934533 ,  0.37712714, -0.92120093, -0.8564395 ],
        [ 0.22940499, -1.

In [53]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure2_4th.h5')

In [54]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_3(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_30 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 105us/step - loss: 0.0075 - val_loss: 0.0046
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0072 - val_loss: 0.0055
Epoch 71/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0076 - val_loss: 0.0059
Epoch 72/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0086 - val_loss: 0.0071
Epoch 73/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0092 - val_loss: 0.0045
Epoch 74/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0079 - val_loss: 0.0047
Epoch 75/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0075 - val_loss: 0.0048
Epoch 76/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0074 - val_loss: 0.0046
Epoch 77/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0071 - val_loss: 0.0046
Epoch 78/1000
143/143 [==============================] - 0s 110us/step

143/143 [==============================] - 0s 112us/step - loss: 0.0065 - val_loss: 0.0054
Epoch 147/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0062 - val_loss: 0.0039
Epoch 148/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0067 - val_loss: 0.0056
Epoch 149/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0067 - val_loss: 0.0044
Epoch 150/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0058 - val_loss: 0.0052
Epoch 151/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0062 - val_loss: 0.0046
Epoch 152/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0060 - val_loss: 0.0045
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0063 - val_loss: 0.0049
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0059 - val_loss: 0.0042
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 106us/step - loss: 0.0063 - val_loss: 0.0054
Epoch 224/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0053 - val_loss: 0.0036
Epoch 225/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0051 - val_loss: 0.0040
Epoch 227/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0052 - val_loss: 0.0046
Epoch 228/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0050 - val_loss: 0.0042
Epoch 229/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0051 - val_loss: 0.0041
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0052 - val_loss: 0.0051
Epoch 231/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0051 - val_loss: 0.0041
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0044
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0037
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0048
Epoch 303/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0048 - val_loss: 0.0040
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0045
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0048 - val_loss: 0.0043
Epoch 306/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0046 - val_loss: 0.0043
Epoch 307/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0045 - val_loss: 0.0038
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0049 - val_loss: 0.0040
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0033
Epoch 763/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0021 - val_loss: 0.0026
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0029
Epoch 765/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0020 - val_loss: 0.0030
Epoch 766/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0018 - val_loss: 0.0030
Epoch 767/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0025 - val_loss: 0.0027
Epoch 768/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0020 - val_loss: 0.0041
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0034
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0031
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 104us/step - loss: 0.0018 - val_loss: 0.0031
Epoch 840/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0020 - val_loss: 0.0029
Epoch 841/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 842/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0019 - val_loss: 0.0034
Epoch 843/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0016 - val_loss: 0.0030
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0036
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0035
Epoch 846/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0041
Epoch 847/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0018 - val_loss: 0.0029
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0030
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0030
Epoch 918/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0014 - val_loss: 0.0034
Epoch 919/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0015 - val_loss: 0.0039
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0038
Epoch 921/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0027
Epoch 922/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0021 - val_loss: 0.0029
Epoch 923/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0016 - val_loss: 0.0032
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0041
Epoch 925/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 111us/step - loss: 0.0020 - val_loss: 0.0030
Epoch 994/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0018 - val_loss: 0.0050
Epoch 995/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0015 - val_loss: 0.0031
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0027
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 998/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0014 - val_loss: 0.0033
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0034
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0029
0.012176395393908024


[array([[ 0.71036226, -0.10302917, -0.9189991 ,  0.39777452,  0.19584052],
        [ 0.22109173,  0.65664095, -0.44598335, -0.6136856 ,  0.08573424],
        [-0.9590096 , -0.37375563,  0.20607613, -0.27657562, -0.19846945],
        [-0.23571002, -0.65722716,  0.03596286,  0.185982  ,  0.01001965],
        [-0.27159145, -0.06250049,  0.32582757, -0.05341841,  0.5017336 ],
        [ 0.89097255,  1.0554409 , -1.0225809 , -0.5462747 ,  0.29750708],
        [-0.1793804 , -0.7214401 , -0.65326005, -0.15002458, -0.3456803 ],
        [-0.02749527, -0.17999767, -0.18576503,  0.12821677, -0.22089916],
        [-0.557536  , -0.24457613,  0.7360202 , -0.42238715, -0.5669123 ],
        [ 0.6126669 ,  0.17354855, -0.8219064 , -0.21962704, -0.16128862],
        [-0.18131714, -0.2282063 ,  0.33201933, -0.3282481 ,  0.04786977],
        [ 0.29796433,  0.6119303 ,  1.9135824 ,  1.581173  ,  0.13115884],
        [ 0.24357021,  0.51009697,  0.2211002 ,  0.6264713 , -0.05434312],
        [ 1.5793031 ,  0.

In [55]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure3_4th.h5')

In [56]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_4(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_31 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_32 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 105us/step - loss: 0.0070 - val_loss: 0.0041
Epoch 70/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0067 - val_loss: 0.0043
Epoch 71/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0064 - val_loss: 0.0042
Epoch 72/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0065 - val_loss: 0.0044
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0069 - val_loss: 0.0042
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0042
Epoch 75/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0075 - val_loss: 0.0042
Epoch 76/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0065 - val_loss: 0.0042
Epoch 77/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0061 - val_loss: 0.0045
Epoch 78/1000
143/143 [==============================] - 0s 98us/step -

143/143 [==============================] - 0s 115us/step - loss: 0.0044 - val_loss: 0.0037
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0033
Epoch 148/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0040 - val_loss: 0.0035
Epoch 149/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0040 - val_loss: 0.0034
Epoch 150/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0046 - val_loss: 0.0065
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0059 - val_loss: 0.0039
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0046
Epoch 153/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0040 - val_loss: 0.0033
Epoch 154/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0037 - val_loss: 0.0048
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 110us/step - loss: 0.0030 - val_loss: 0.0031
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0043
Epoch 225/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0033 - val_loss: 0.0029
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0032
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0030 - val_loss: 0.0029
Epoch 228/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0031 - val_loss: 0.0055
Epoch 229/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0040 - val_loss: 0.0028
Epoch 230/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0031 - val_loss: 0.0029
Epoch 231/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0031 - val_loss: 0.0039
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0027
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0035
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0029
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0028
Epoch 304/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0027 - val_loss: 0.0039
Epoch 305/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0029 - val_loss: 0.0031
Epoch 306/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0029 - val_loss: 0.0030
Epoch 307/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0031 - val_loss: 0.0039
Epoch 308/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0026 - val_loss: 0.0026
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0033
Epoch 763/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0015 - val_loss: 0.0036
Epoch 764/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 765/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0034
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0035
Epoch 767/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0035
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 769/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0015 - val_loss: 0.0052
Epoch 770/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0019 - val_loss: 0.0035
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 840/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0014 - val_loss: 0.0034
Epoch 841/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0034
Epoch 842/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0016 - val_loss: 0.0054
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0052
Epoch 844/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 845/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 846/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0040
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0038
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0036
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0039
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 919/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 920/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0015 - val_loss: 0.0038
Epoch 921/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0012 - val_loss: 0.0047
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0044
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0043
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0036
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0044
Epoch 994/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0013 - val_loss: 0.0043
Epoch 995/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0012 - val_loss: 0.0044
Epoch 996/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0012 - val_loss: 0.0045
Epoch 997/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 998/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0011 - val_loss: 0.0043
Epoch 999/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0012 - val_loss: 0.0060
Epoch 1000/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0017 - val_loss: 0.0056
0.0053664036095142365


[array([[-1.04212761e-01, -1.54211313e-01, -1.17019132e-01,
         -6.55142665e-01, -3.82029384e-01,  2.49525383e-01,
         -1.02312326e+00, -1.21020891e-01, -1.64541692e-01,
         -6.66790485e-01],
        [ 2.94268757e-01, -3.35404098e-01, -2.19164327e-01,
         -8.25215816e-01,  1.31606713e-01,  2.81991102e-02,
         -4.01895463e-01,  4.62316006e-01, -3.67710501e-01,
          2.19327837e-01],
        [ 1.12858322e-02, -3.06483328e-01, -2.50414938e-01,
         -6.53700531e-01,  2.95567751e-01, -3.19169968e-01,
          2.66283661e-01, -1.51329115e-01,  3.18544686e-01,
          1.91502199e-01],
        [-5.20174623e-01, -7.16784120e-01,  1.40636647e+00,
         -3.64394754e-01, -1.51088035e+00,  9.48521018e-01,
          1.01856577e+00, -8.04215372e-01, -4.15553421e-01,
         -2.67472625e-01],
        [ 2.66922176e-01, -7.15288222e-02, -1.44352570e-01,
          3.47454041e-01, -4.21235412e-01,  3.19446437e-02,
          2.85900712e-01, -3.35828245e-01, -4.216660

In [57]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure4_4th.h5')

In [58]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_5(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_33 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_34 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 11        
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 110us/step - loss: 0.0061 - val_loss: 0.0058
Epoch 70/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0072 - val_loss: 0.0080
Epoch 71/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0068 - val_loss: 0.0057
Epoch 72/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0062 - val_loss: 0.0054
Epoch 73/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0057 - val_loss: 0.0053
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0056
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0067
Epoch 76/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0057 - val_loss: 0.0054
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0055
Epoch 78/1000
143/143 [==============================] - 0s 107us/ste

143/143 [==============================] - 0s 105us/step - loss: 0.0045 - val_loss: 0.0083
Epoch 147/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0054 - val_loss: 0.0046
Epoch 148/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0049 - val_loss: 0.0056
Epoch 149/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0047 - val_loss: 0.0054
Epoch 150/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0044 - val_loss: 0.0057
Epoch 151/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0044
Epoch 152/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0043 - val_loss: 0.0042
Epoch 153/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0042 - val_loss: 0.0051
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0067
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0040
Epoch 224/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0035 - val_loss: 0.0052
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0041
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0051
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0047
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0038
Epoch 229/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0022 - val_loss: 0.0042
Epoch 230/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0023 - val_loss: 0.0055
Epoch 231/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0029 - val_loss: 0.0046
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0019 - val_loss: 0.0040
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0038
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0035
Epoch 303/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0023 - val_loss: 0.0064
Epoch 304/1000
143/143 [==============================] - 0s 140us/step - loss: 0.0025 - val_loss: 0.0038
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0033
Epoch 306/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0018 - val_loss: 0.0033
Epoch 307/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0035
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0035
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0041
Epoch 763/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 764/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0042
Epoch 765/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0038
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0054
Epoch 767/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 768/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 769/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0012 - val_loss: 0.0043
Epoch 770/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0013 - val_loss: 0.0041
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 106us/step - loss: 9.8073e-04 - val_loss: 0.0044
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0050
Epoch 841/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 842/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0043
Epoch 843/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0039
Epoch 845/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 846/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0011 - val_loss: 0.0048
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0039
Epoch 848/1000
143/143 [==============================] -

143/143 [==============================] - 0s 105us/step - loss: 0.0010 - val_loss: 0.0047
Epoch 916/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0011 - val_loss: 0.0047
Epoch 917/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0011 - val_loss: 0.0043
Epoch 918/1000
143/143 [==============================] - 0s 105us/step - loss: 9.1703e-04 - val_loss: 0.0048
Epoch 919/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0010 - val_loss: 0.0045
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0044
Epoch 921/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0011 - val_loss: 0.0042
Epoch 922/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0010 - val_loss: 0.0042
Epoch 923/1000
143/143 [==============================] - 0s 104us/step - loss: 9.3054e-04 - val_loss: 0.0049
Epoch 924/1000
143/143 [==============================

143/143 [==============================] - 0s 112us/step - loss: 8.9740e-04 - val_loss: 0.0040
Epoch 992/1000
143/143 [==============================] - 0s 105us/step - loss: 9.9954e-04 - val_loss: 0.0050
Epoch 993/1000
143/143 [==============================] - 0s 105us/step - loss: 9.9605e-04 - val_loss: 0.0046
Epoch 994/1000
143/143 [==============================] - 0s 113us/step - loss: 9.6928e-04 - val_loss: 0.0043
Epoch 995/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0010 - val_loss: 0.0045
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 9.2048e-04 - val_loss: 0.0054
Epoch 997/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0010 - val_loss: 0.0048
Epoch 998/1000
143/143 [==============================] - 0s 111us/step - loss: 9.1856e-04 - val_loss: 0.0051
Epoch 999/1000
143/143 [==============================] - 0s 114us/step - loss: 8.6863e-04 - val_loss: 0.0042
Epoch 1000/1000
143/143 [========

[array([[-1.00431487e-01,  1.21067889e-01,  4.09593701e-01,
          2.40309685e-01,  4.03337516e-02, -7.69879818e-01,
          1.73465982e-01, -1.56160223e+00,  9.49560106e-02,
          1.63260773e-01],
        [ 4.40749466e-01, -1.25971520e+00,  2.63565928e-01,
         -3.05983484e-01, -1.07567355e-01, -2.62014776e-01,
          3.20910156e-01,  7.81092346e-01, -1.52180493e-01,
          4.75504011e-01],
        [ 5.03566384e-01,  3.82459313e-01,  1.12733006e-01,
         -3.24534923e-01, -1.89240366e-01,  3.56999099e-01,
          1.36057049e-01,  7.89302409e-01, -3.65474999e-01,
          2.27050930e-01],
        [ 1.15214497e-01, -2.54611325e+00,  7.75862396e-01,
          2.33249158e-01, -2.16309384e-01, -7.78955460e-01,
         -1.24643862e-01,  3.59998226e-01,  1.13832392e-02,
          1.98465526e-01],
        [ 2.11324230e-01, -3.31838056e-02,  4.37098324e-01,
          2.49756649e-01,  6.44292012e-02, -1.24731712e-01,
          5.26124895e-01,  2.32355878e-01,  1.519753

In [59]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure5_4th.h5')

In [60]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_6(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_35 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_36 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 16        
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 111us/step - loss: 0.0058 - val_loss: 0.0052
Epoch 70/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0060 - val_loss: 0.0051
Epoch 71/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0057 - val_loss: 0.0065
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0065 - val_loss: 0.0051
Epoch 73/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0062 - val_loss: 0.0055
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0058 - val_loss: 0.0042
Epoch 75/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0059 - val_loss: 0.0051
Epoch 76/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0056 - val_loss: 0.0043
Epoch 77/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0057 - val_loss: 0.0043
Epoch 78/1000
143/143 [==============================] - 0s 105us/ste

143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0038
Epoch 147/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0043 - val_loss: 0.0043
Epoch 148/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0043 - val_loss: 0.0044
Epoch 149/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 150/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0040 - val_loss: 0.0044
Epoch 151/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 152/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0043 - val_loss: 0.0037
Epoch 153/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0042 - val_loss: 0.0061
Epoch 154/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0044 - val_loss: 0.0046
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0032 - val_loss: 0.0043
Epoch 224/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0031 - val_loss: 0.0049
Epoch 225/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0035 - val_loss: 0.0060
Epoch 226/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0037 - val_loss: 0.0053
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0040 - val_loss: 0.0053
Epoch 228/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0038 - val_loss: 0.0049
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0044
Epoch 230/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0052
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0041
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0045
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0049
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0049
Epoch 303/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0023 - val_loss: 0.0045
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0046
Epoch 305/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0025 - val_loss: 0.0061
Epoch 306/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0026 - val_loss: 0.0041
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0042
Epoch 308/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0022 - val_loss: 0.0046
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0056
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0049
Epoch 764/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0049
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0047
Epoch 766/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0052
Epoch 767/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0059
Epoch 768/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0048
Epoch 769/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0052
Epoch 770/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0011 - val_loss: 0.0053
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 115us/step - loss: 0.0012 - val_loss: 0.0059
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0055
Epoch 841/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0011 - val_loss: 0.0053
Epoch 842/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0011 - val_loss: 0.0050
Epoch 843/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0016 - val_loss: 0.0050
Epoch 844/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0014 - val_loss: 0.0068
Epoch 845/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0016 - val_loss: 0.0064
Epoch 846/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0018 - val_loss: 0.0050
Epoch 847/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0016 - val_loss: 0.0048
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0056
Epoch 917/1000
143/143 [==============================] - 0s 111us/step - loss: 9.8272e-04 - val_loss: 0.0056
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0055
Epoch 919/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0011 - val_loss: 0.0056
Epoch 920/1000
143/143 [==============================] - 0s 113us/step - loss: 9.7225e-04 - val_loss: 0.0054
Epoch 921/1000
143/143 [==============================] - 0s 106us/step - loss: 9.3801e-04 - val_loss: 0.0057
Epoch 922/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0010 - val_loss: 0.0057
Epoch 923/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0012 - val_loss: 0.0069
Epoch 924/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0012 - val_loss: 0.0064
Epoch 925/1000
143/143 [=========================

143/143 [==============================] - 0s 106us/step - loss: 0.0013 - val_loss: 0.0059
Epoch 993/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0011 - val_loss: 0.0052
Epoch 994/1000
143/143 [==============================] - 0s 105us/step - loss: 9.7538e-04 - val_loss: 0.0060
Epoch 995/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0012 - val_loss: 0.0067
Epoch 996/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0064
Epoch 997/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0015 - val_loss: 0.0055
Epoch 998/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0017 - val_loss: 0.0062
Epoch 999/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0019 - val_loss: 0.0056
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0067
0.004818336572498083


[array([[-9.04928327e-01,  5.18170118e-01, -1.55771365e-02,
          2.80922651e-01,  8.63481998e-01,  8.31255317e-01,
         -4.87107784e-01,  3.48561645e-01,  4.08805162e-01,
         -1.61818862e-01],
        [-5.41140556e-01,  2.73890823e-01, -2.36904368e-01,
          9.03675407e-02, -6.92040741e-01,  5.78446507e-01,
          7.34928489e-01,  2.72814534e-03, -7.65474141e-02,
          1.16567202e-01],
        [ 1.06496930e+00,  5.69713265e-02,  8.67590308e-02,
         -3.16236109e-01, -5.60738205e-04,  2.24754378e-01,
          2.02225015e-01, -2.42828846e-01,  4.09718186e-01,
          7.56944567e-02],
        [ 1.09439421e+00,  2.48776689e-01,  3.83852899e-01,
         -1.11993238e-01,  8.93789887e-01, -2.98329797e-02,
         -3.77590984e-01, -2.39643138e-02,  4.39037055e-01,
         -2.91052669e-01],
        [ 1.20551579e-01, -1.74420238e-01, -2.52269179e-01,
          5.90089142e-01,  2.11817101e-01, -3.25537711e-01,
         -4.08314496e-01, -3.05178523e-01,  2.189272

In [61]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure6_4th.h5')

In [62]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_1(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_37 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_56 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_38 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 6         
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 12ms/step - loss: 37.25

Epoch 71/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0179 - val_loss: 0.0077
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0172 - val_loss: 0.0079
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0175 - val_loss: 0.0085
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0183 - val_loss: 0.0092
Epoch 75/1000
32/32 [==============================] - 0s 159us/step - loss: 0.0193 - val_loss: 0.0098
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0200 - val_loss: 0.0101
Epoch 77/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0203 - val_loss: 0.0099
Epoch 78/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0200 - val_loss: 0.0094
Epoch 79/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0191 - val_loss: 0.0085
Epoch 80/1000
32/32 [==============================] - 0s 157us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0084 - val_loss: 0.0017
Epoch 152/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0083 - val_loss: 0.0017
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0083 - val_loss: 0.0017
Epoch 154/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0082 - val_loss: 0.0017
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0082 - val_loss: 0.0017
Epoch 156/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0081 - val_loss: 0.0016
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0081 - val_loss: 0.0016
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0080 - val_loss: 0.0016
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0080 - val_loss: 0.0016
Epoch 160/1000
32/32 [==============================] - 0s 125us

Epoch 230/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0055 - val_loss: 8.5686e-04
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0055 - val_loss: 8.5049e-04
Epoch 232/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0055 - val_loss: 8.4419e-04
Epoch 233/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0055 - val_loss: 8.3795e-04
Epoch 234/1000
32/32 [==============================] - 0s 249us/step - loss: 0.0054 - val_loss: 8.3181e-04
Epoch 235/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0054 - val_loss: 8.2573e-04
Epoch 236/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0054 - val_loss: 8.1972e-04
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0054 - val_loss: 8.1381e-04
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0053 - val_loss: 8.0798e-04
Epoch 239/1000
32/32 [======

Epoch 306/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 5.2980e-04
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 5.2699e-04
Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 5.2424e-04
Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 5.2151e-04
Epoch 310/1000
32/32 [==============================] - 0s 138us/step - loss: 0.0040 - val_loss: 5.1880e-04
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 5.1613e-04
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 5.1348e-04
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0039 - val_loss: 5.1085e-04
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 5.0825e-04
Epoch 315/1000
32/32 [======

...

Epoch 762/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0010 - val_loss: 1.4089e-04
Epoch 763/1000
32/32 [==============================] - 0s 132us/step - loss: 0.0010 - val_loss: 1.4054e-04
Epoch 764/1000
32/32 [==============================] - 0s 151us/step - loss: 9.9751e-04 - val_loss: 1.4021e-04
Epoch 765/1000
32/32 [==============================] - 0s 152us/step - loss: 9.9502e-04 - val_loss: 1.3987e-04
Epoch 766/1000
32/32 [==============================] - 0s 142us/step - loss: 9.9253e-04 - val_loss: 1.3953e-04
Epoch 767/1000
32/32 [==============================] - 0s 155us/step - loss: 9.9006e-04 - val_loss: 1.3919e-04
Epoch 768/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8759e-04 - val_loss: 1.3886e-04
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8512e-04 - val_loss: 1.3853e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8268e-04 - val_loss: 1.3819e-04


Epoch 836/1000
32/32 [==============================] - 0s 134us/step - loss: 8.3681e-04 - val_loss: 1.1805e-04
Epoch 837/1000
32/32 [==============================] - 0s 155us/step - loss: 8.3483e-04 - val_loss: 1.1777e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 8.3285e-04 - val_loss: 1.1749e-04
Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 8.3087e-04 - val_loss: 1.1722e-04
Epoch 840/1000
32/32 [==============================] - 0s 128us/step - loss: 8.2890e-04 - val_loss: 1.1695e-04
Epoch 841/1000
32/32 [==============================] - 0s 136us/step - loss: 8.2694e-04 - val_loss: 1.1667e-04
Epoch 842/1000
32/32 [==============================] - 0s 156us/step - loss: 8.2498e-04 - val_loss: 1.1640e-04
Epoch 843/1000
32/32 [==============================] - 0s 142us/step - loss: 8.2302e-04 - val_loss: 1.1613e-04
Epoch 844/1000
32/32 [==============================] - 0s 130us/step - loss: 8.2108e-04 - val_loss: 1.1

Epoch 910/1000
32/32 [==============================] - 0s 156us/step - loss: 7.0481e-04 - val_loss: 9.9617e-05
Epoch 911/1000
32/32 [==============================] - 0s 168us/step - loss: 7.0322e-04 - val_loss: 9.9397e-05
Epoch 912/1000
32/32 [==============================] - 0s 156us/step - loss: 7.0163e-04 - val_loss: 9.9172e-05
Epoch 913/1000
32/32 [==============================] - 0s 161us/step - loss: 7.0005e-04 - val_loss: 9.8955e-05
Epoch 914/1000
32/32 [==============================] - 0s 118us/step - loss: 6.9847e-04 - val_loss: 9.8730e-05
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 6.9690e-04 - val_loss: 9.8510e-05
Epoch 916/1000
32/32 [==============================] - 0s 154us/step - loss: 6.9533e-04 - val_loss: 9.8294e-05
Epoch 917/1000
32/32 [==============================] - 0s 133us/step - loss: 6.9376e-04 - val_loss: 9.8070e-05
Epoch 918/1000
32/32 [==============================] - 0s 131us/step - loss: 6.9220e-04 - val_loss: 9.7

Epoch 984/1000
32/32 [==============================] - 0s 136us/step - loss: 5.9850e-04 - val_loss: 8.4689e-05
Epoch 985/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9721e-04 - val_loss: 8.4504e-05
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9592e-04 - val_loss: 8.4325e-05
Epoch 987/1000
32/32 [==============================] - 0s 118us/step - loss: 5.9465e-04 - val_loss: 8.4146e-05
Epoch 988/1000
32/32 [==============================] - 0s 144us/step - loss: 5.9337e-04 - val_loss: 8.3970e-05
Epoch 989/1000
32/32 [==============================] - 0s 144us/step - loss: 5.9209e-04 - val_loss: 8.3781e-05
Epoch 990/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9082e-04 - val_loss: 8.3604e-05
Epoch 991/1000
32/32 [==============================] - 0s 125us/step - loss: 5.8955e-04 - val_loss: 8.3429e-05
Epoch 992/1000
32/32 [==============================] - 0s 124us/step - loss: 5.8829e-04 - val_loss: 8.3

[array([[-0.9547809 , -0.70075583,  0.93348825, -1.0377702 ,  1.1184882 ],
        [ 0.06112464, -0.0985145 , -0.6434552 ,  1.1727086 , -0.5420046 ],
        [ 0.2241915 , -1.2524726 , -1.3355135 , -0.8867324 ,  0.45765135]],
       dtype=float32),
 array([-0.82537836, -0.68842566,  0.67985415,  0.8553519 , -0.60817665],
       dtype=float32),
 array([[-0.323142  ,  1.1352091 , -0.12353183, -0.2643727 ,  0.64439124],
        [-0.46887562, -0.00236923,  0.74908805, -0.37208283,  0.2561972 ],
        [ 0.79080075,  0.30135262, -0.48065734, -0.31345648,  0.00156789],
        [-0.4564334 ,  0.14380543, -0.68713963, -0.5933642 , -0.21551353],
        [-1.1562613 ,  0.17627661,  0.01170437,  0.01910819,  0.76695037]],
       dtype=float32),
 array([ 0.8943569 ,  0.11826516, -0.8373644 ,  0.8464892 , -0.8707357 ],
       dtype=float32),
 array([[ 1.2984133 ],
        [-0.10585112],
        [-0.6966867 ],
        [ 0.6507614 ],
        [-0.90641415]], dtype=float32),
 array([0.929705], dtype=f

In [63]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure1_4th.h5')

In [64]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_2(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_39 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_40 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 12ms/step - loss: 34.82

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0106 - val_loss: 0.0156
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0108 - val_loss: 0.0140
Epoch 73/1000
32/32 [==============================] - 0s 136us/step - loss: 0.0109 - val_loss: 0.0129
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0107 - val_loss: 0.0120
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0104 - val_loss: 0.0112
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0098 - val_loss: 0.0105
Epoch 77/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0091 - val_loss: 0.0098
Epoch 78/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0085 - val_loss: 0.0093
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0080 - val_loss: 0.0090
Epoch 80/1000
32/32 [==============================] - 0s 122us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 168us/step - loss: 0.0041 - val_loss: 0.0046
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0041 - val_loss: 0.0045
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 0.0045
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 0.0044
Epoch 155/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0040 - val_loss: 0.0044
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 0.0043
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0043
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0042
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0042
Epoch 160/1000
32/32 [==============================] - 0s 141us

Epoch 230/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0025 - val_loss: 0.0030
Epoch 231/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 232/1000
32/32 [==============================] - 0s 118us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 233/1000
32/32 [==============================] - 0s 148us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 236/1000
32/32 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0029
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0029
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0023
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0023
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0023
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0023
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0023
Epoch 314/1000
32/32 [==============================] - 0s 158us/step - loss: 0.0016 - val_loss: 0.0023
Epoch 315/1000
32/32 [==============================] - 0s 141us/step - loss: 0.0016 - val_loss: 0.0023
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0022
Epoch 317/1000
32/32 [==============================] - 0s 142us/step - loss: 0.0015 - val_loss: 0.0022
Epoch 318/1000
32/32 [==============================] - 0s 132us

...

Epoch 766/1000
32/32 [==============================] - 0s 170us/step - loss: 3.8466e-04 - val_loss: 7.8766e-04
Epoch 767/1000
32/32 [==============================] - 0s 157us/step - loss: 3.8390e-04 - val_loss: 7.8625e-04
Epoch 768/1000
32/32 [==============================] - 0s 129us/step - loss: 3.8313e-04 - val_loss: 7.8485e-04
Epoch 769/1000
32/32 [==============================] - 0s 181us/step - loss: 3.8236e-04 - val_loss: 7.8344e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8160e-04 - val_loss: 7.8204e-04
Epoch 771/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8084e-04 - val_loss: 7.8063e-04
Epoch 772/1000
32/32 [==============================] - 0s 187us/step - loss: 3.8008e-04 - val_loss: 7.7926e-04
Epoch 773/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7932e-04 - val_loss: 7.7787e-04
Epoch 774/1000
32/32 [==============================] - 0s 124us/step - loss: 3.7857e-04 - val_loss: 7.7

Epoch 840/1000
32/32 [==============================] - 0s 136us/step - loss: 3.3354e-04 - val_loss: 6.9334e-04
Epoch 841/1000
32/32 [==============================] - 0s 155us/step - loss: 3.3292e-04 - val_loss: 6.9219e-04
Epoch 842/1000
32/32 [==============================] - 0s 125us/step - loss: 3.3231e-04 - val_loss: 6.9103e-04
Epoch 843/1000
32/32 [==============================] - 0s 156us/step - loss: 3.3169e-04 - val_loss: 6.8989e-04
Epoch 844/1000
32/32 [==============================] - 0s 156us/step - loss: 3.3108e-04 - val_loss: 6.8873e-04
Epoch 845/1000
32/32 [==============================] - 0s 125us/step - loss: 3.3046e-04 - val_loss: 6.8761e-04
Epoch 846/1000
32/32 [==============================] - 0s 154us/step - loss: 3.2985e-04 - val_loss: 6.8650e-04
Epoch 847/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2925e-04 - val_loss: 6.8536e-04
Epoch 848/1000
32/32 [==============================] - 0s 153us/step - loss: 3.2864e-04 - val_loss: 6.8

Epoch 914/1000
32/32 [==============================] - 0s 123us/step - loss: 2.9198e-04 - val_loss: 6.1591e-04
Epoch 915/1000
32/32 [==============================] - 0s 139us/step - loss: 2.9147e-04 - val_loss: 6.1495e-04
Epoch 916/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9096e-04 - val_loss: 6.1401e-04
Epoch 917/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9045e-04 - val_loss: 6.1307e-04
Epoch 918/1000
32/32 [==============================] - 0s 125us/step - loss: 2.8995e-04 - val_loss: 6.1214e-04
Epoch 919/1000
32/32 [==============================] - 0s 124us/step - loss: 2.8945e-04 - val_loss: 6.1119e-04
Epoch 920/1000
32/32 [==============================] - 0s 153us/step - loss: 2.8894e-04 - val_loss: 6.1027e-04
Epoch 921/1000
32/32 [==============================] - 0s 171us/step - loss: 2.8844e-04 - val_loss: 6.0933e-04
Epoch 922/1000
32/32 [==============================] - 0s 124us/step - loss: 2.8794e-04 - val_loss: 6.0

Epoch 988/1000
32/32 [==============================] - 0s 124us/step - loss: 2.5753e-04 - val_loss: 5.5154e-04
Epoch 989/1000
32/32 [==============================] - 0s 154us/step - loss: 2.5711e-04 - val_loss: 5.5076e-04
Epoch 990/1000
32/32 [==============================] - 0s 144us/step - loss: 2.5668e-04 - val_loss: 5.4995e-04
Epoch 991/1000
32/32 [==============================] - 0s 131us/step - loss: 2.5625e-04 - val_loss: 5.4916e-04
Epoch 992/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5583e-04 - val_loss: 5.4836e-04
Epoch 993/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5542e-04 - val_loss: 5.4758e-04
Epoch 994/1000
32/32 [==============================] - 0s 148us/step - loss: 2.5499e-04 - val_loss: 5.4680e-04
Epoch 995/1000
32/32 [==============================] - 0s 124us/step - loss: 2.5458e-04 - val_loss: 5.4601e-04
Epoch 996/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5415e-04 - val_loss: 5.4

[array([[ 0.7391709 , -0.95250016, -0.93333143, -1.2681283 ,  0.82242435],
        [ 0.35123497,  0.2080682 ,  0.46274352,  1.087654  , -1.368843  ],
        [ 1.3190284 , -0.64562   ,  1.0021868 , -0.45566693,  0.38974926]],
       dtype=float32),
 array([ 0.68261176,  0.64783657, -0.68546295,  0.60318595, -0.62538654],
       dtype=float32),
 array([[ 8.3876893e-02,  2.8080171e-01,  3.3434358e-01, -7.2249514e-01,
          7.1030885e-02, -4.1968998e-01, -7.1529275e-01, -4.8162454e-01,
         -9.3180215e-01,  9.3115681e-01],
        [-1.0784489e-01,  6.0161781e-01, -8.7076610e-01, -5.4619873e-01,
          3.0964324e-01, -4.4019021e-02, -6.7870235e-01, -5.0568998e-01,
         -3.3278811e-01, -1.8494247e-01],
        [ 1.8683141e-01, -8.4142870e-01,  2.7187839e-01,  5.8998793e-01,
          5.1964629e-01,  6.9054466e-01, -2.6067179e-03,  6.0137808e-01,
          8.5432225e-01, -7.0975035e-01],
        [ 3.3683306e-01,  5.9547400e-01,  4.5786655e-01, -6.4976972e-01,
          1.82267

In [65]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure2_4th.h5')

In [66]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_3(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_41 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_62 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_42 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 16        
Total params: 126
Trainable params: 126
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 12ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0141 - val_loss: 0.0040
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0153 - val_loss: 0.0047
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0169 - val_loss: 0.0051
Epoch 74/1000
32/32 [==============================] - 0s 159us/step - loss: 0.0182 - val_loss: 0.0050
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0187 - val_loss: 0.0045
Epoch 76/1000
32/32 [==============================] - 0s 149us/step - loss: 0.0183 - val_loss: 0.0038
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0172 - val_loss: 0.0030
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0155 - val_loss: 0.0024
Epoch 79/1000
32/32 [==============================] - 0s 128us/step - loss: 0.0139 - val_loss: 0.0020
Epoch 80/1000
32/32 [==============================] - 0s 155us/step - lo

Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0061 - val_loss: 6.7222e-04
Epoch 151/1000
32/32 [==============================] - 0s 181us/step - loss: 0.0061 - val_loss: 6.6640e-04
Epoch 152/1000
32/32 [==============================] - 0s 158us/step - loss: 0.0061 - val_loss: 6.5888e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0060 - val_loss: 6.5007e-04
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0060 - val_loss: 6.4047e-04
Epoch 155/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0059 - val_loss: 6.3051e-04
Epoch 156/1000
32/32 [==============================] - 0s 132us/step - loss: 0.0059 - val_loss: 6.2051e-04
Epoch 157/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0059 - val_loss: 6.1076e-04
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0058 - val_loss: 6.0134e-04
Epoch 159/1000
32/32 [======

Epoch 226/1000
32/32 [==============================] - 0s 136us/step - loss: 0.0039 - val_loss: 2.8277e-04
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 2.7998e-04
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 2.7720e-04
Epoch 229/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0039 - val_loss: 2.7445e-04
Epoch 230/1000
32/32 [==============================] - 0s 135us/step - loss: 0.0039 - val_loss: 2.7175e-04
Epoch 231/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0038 - val_loss: 2.6909e-04
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 2.6648e-04
Epoch 233/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0038 - val_loss: 2.6394e-04
Epoch 234/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0038 - val_loss: 2.6145e-04
Epoch 235/1000
32/32 [======

Epoch 302/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0028 - val_loss: 1.4902e-04
Epoch 303/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0028 - val_loss: 1.4797e-04
Epoch 304/1000
32/32 [==============================] - 0s 128us/step - loss: 0.0028 - val_loss: 1.4695e-04
Epoch 305/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0028 - val_loss: 1.4594e-04
Epoch 306/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0028 - val_loss: 1.4494e-04
Epoch 307/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0028 - val_loss: 1.4396e-04
Epoch 308/1000
32/32 [==============================] - 0s 151us/step - loss: 0.0028 - val_loss: 1.4299e-04
Epoch 309/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0028 - val_loss: 1.4203e-04
Epoch 310/1000
32/32 [==============================] - 0s 165us/step - loss: 0.0027 - val_loss: 1.4108e-04
Epoch 311/1000
32/32 [======

...

Epoch 756/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0906e-04 - val_loss: 6.0743e-05
Epoch 757/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0721e-04 - val_loss: 6.0703e-05
Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0537e-04 - val_loss: 6.0666e-05
Epoch 759/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0351e-04 - val_loss: 6.0626e-05
Epoch 760/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0168e-04 - val_loss: 6.0590e-05
Epoch 761/1000
32/32 [==============================] - 0s 124us/step - loss: 7.9985e-04 - val_loss: 6.0557e-05
Epoch 762/1000
32/32 [==============================] - 0s 124us/step - loss: 7.9803e-04 - val_loss: 6.0530e-05
Epoch 763/1000
32/32 [==============================] - 0s 156us/step - loss: 7.9620e-04 - val_loss: 6.0485e-05
Epoch 764/1000
32/32 [==============================] - 0s 124us/step - loss: 7.9440e-04 - val_loss: 6.0

Epoch 830/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8441e-04 - val_loss: 5.8071e-05
Epoch 831/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8288e-04 - val_loss: 5.8029e-05
Epoch 832/1000
32/32 [==============================] - 0s 162us/step - loss: 6.8135e-04 - val_loss: 5.7993e-05
Epoch 833/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7983e-04 - val_loss: 5.7956e-05
Epoch 834/1000
32/32 [==============================] - 0s 150us/step - loss: 6.7832e-04 - val_loss: 5.7912e-05
Epoch 835/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7681e-04 - val_loss: 5.7883e-05
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7530e-04 - val_loss: 5.7841e-05
Epoch 837/1000
32/32 [==============================] - 0s 187us/step - loss: 6.7380e-04 - val_loss: 5.7799e-05
Epoch 838/1000
32/32 [==============================] - 0s 218us/step - loss: 6.7229e-04 - val_loss: 5.7

Epoch 904/1000
32/32 [==============================] - 0s 145us/step - loss: 5.8114e-04 - val_loss: 5.5276e-05
Epoch 905/1000
32/32 [==============================] - 0s 122us/step - loss: 5.7987e-04 - val_loss: 5.5243e-05
Epoch 906/1000
32/32 [==============================] - 0s 143us/step - loss: 5.7860e-04 - val_loss: 5.5205e-05
Epoch 907/1000
32/32 [==============================] - 0s 125us/step - loss: 5.7735e-04 - val_loss: 5.5168e-05
Epoch 908/1000
32/32 [==============================] - 0s 155us/step - loss: 5.7608e-04 - val_loss: 5.5128e-05
Epoch 909/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7482e-04 - val_loss: 5.5093e-05
Epoch 910/1000
32/32 [==============================] - 0s 150us/step - loss: 5.7357e-04 - val_loss: 5.5061e-05
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7232e-04 - val_loss: 5.5020e-05
Epoch 912/1000
32/32 [==============================] - 0s 126us/step - loss: 5.7107e-04 - val_loss: 5.4

Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9497e-04 - val_loss: 5.2437e-05
Epoch 979/1000
32/32 [==============================] - 0s 125us/step - loss: 4.9391e-04 - val_loss: 5.2397e-05
Epoch 980/1000
32/32 [==============================] - 0s 125us/step - loss: 4.9284e-04 - val_loss: 5.2360e-05
Epoch 981/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9179e-04 - val_loss: 5.2324e-05
Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9073e-04 - val_loss: 5.2285e-05
Epoch 983/1000
32/32 [==============================] - 0s 125us/step - loss: 4.8968e-04 - val_loss: 5.2246e-05
Epoch 984/1000
32/32 [==============================] - 0s 125us/step - loss: 4.8863e-04 - val_loss: 5.2206e-05
Epoch 985/1000
32/32 [==============================] - 0s 124us/step - loss: 4.8758e-04 - val_loss: 5.2173e-05
Epoch 986/1000
32/32 [==============================] - 0s 109us/step - loss: 4.8653e-04 - val_loss: 5.2

[array([[-0.55753225, -0.50120103, -0.7822824 , -0.8092235 , -0.09821057],
        [-0.04113822,  0.52122784,  0.35070112,  0.29268846,  1.1040753 ],
        [-0.7114781 ,  0.63289607,  0.9380625 , -0.1647588 , -0.24742268]],
       dtype=float32),
 array([-0.5337758 , -0.35636297, -0.3303343 ,  0.4020515 ,  0.48349905],
       dtype=float32),
 array([[-4.23586369e-01, -2.01272532e-01, -2.85033286e-01,
          1.62009835e-01,  5.31608760e-01,  1.01405382e-01,
          6.28967524e-01,  1.24211974e-01,  1.55125871e-01,
         -8.08865800e-02, -4.72020686e-01, -5.71879983e-01,
         -7.61063844e-02, -1.42382517e-01,  4.21174496e-01],
        [-2.12677032e-01,  4.92719322e-01,  5.63167810e-01,
          4.49750274e-01,  2.35654294e-01,  6.74845725e-02,
          4.85402554e-01,  4.14017886e-01, -1.65185258e-01,
         -3.83823067e-01,  1.58777798e-03,  3.81127656e-01,
         -8.92196238e-01, -1.73687078e-02,  1.00223444e-01],
        [-2.87828654e-01, -3.38524692e-02,  3.235496

In [67]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure3_4th.h5')

In [68]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_4(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_43 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_44 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 6         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 13ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0342 - val_loss: 0.0235
Epoch 72/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0311 - val_loss: 0.0230
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0280 - val_loss: 0.0231
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0255 - val_loss: 0.0238
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0239 - val_loss: 0.0249
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0231 - val_loss: 0.0263
Epoch 77/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0230 - val_loss: 0.0276
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0232 - val_loss: 0.0285
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0234 - val_loss: 0.0288
Epoch 80/1000
32/32 [==============================] - 0s 164us/step - lo

32/32 [==============================] - 0s 158us/step - loss: 0.0043 - val_loss: 0.0068
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0042 - val_loss: 0.0067
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0042 - val_loss: 0.0066
Epoch 153/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0041 - val_loss: 0.0065
Epoch 154/1000
32/32 [==============================] - 0s 143us/step - loss: 0.0040 - val_loss: 0.0063
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0062
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0061
Epoch 157/1000
32/32 [==============================] - 0s 159us/step - loss: 0.0038 - val_loss: 0.0060
Epoch 158/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0037 - val_loss: 0.0059
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0

Epoch 229/1000
32/32 [==============================] - 0s 123us/step - loss: 0.0013 - val_loss: 0.0017
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0017
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0017
Epoch 232/1000
32/32 [==============================] - 0s 132us/step - loss: 0.0013 - val_loss: 0.0016
Epoch 233/1000
32/32 [==============================] - 0s 150us/step - loss: 0.0012 - val_loss: 0.0016
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0012 - val_loss: 0.0016
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0012 - val_loss: 0.0015
Epoch 236/1000
32/32 [==============================] - 0s 143us/step - loss: 0.0012 - val_loss: 0.0015
Epoch 237/1000
32/32 [==============================] - 0s 129us/step - loss: 0.0012 - val_loss: 0.0015
Epoch 238/1000
32/32 [==============================] - 0s 155us

Epoch 305/1000
32/32 [==============================] - 0s 187us/step - loss: 7.2141e-04 - val_loss: 6.0749e-04
Epoch 306/1000
32/32 [==============================] - 0s 163us/step - loss: 7.1803e-04 - val_loss: 6.0167e-04
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1471e-04 - val_loss: 5.9598e-04
Epoch 308/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1144e-04 - val_loss: 5.9040e-04
Epoch 309/1000
32/32 [==============================] - 0s 131us/step - loss: 7.0822e-04 - val_loss: 5.8494e-04
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 7.0506e-04 - val_loss: 5.7959e-04
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 7.0195e-04 - val_loss: 5.7434e-04
Epoch 312/1000
32/32 [==============================] - 0s 142us/step - loss: 6.9889e-04 - val_loss: 5.6919e-04
Epoch 313/1000
32/32 [==============================] - 0s 148us/step - loss: 6.9588e-04 - val_loss: 5.6

...

Epoch 749/1000
32/32 [==============================] - 0s 141us/step - loss: 3.4382e-04 - val_loss: 1.7566e-04
Epoch 750/1000
32/32 [==============================] - 0s 139us/step - loss: 3.4343e-04 - val_loss: 1.7542e-04
Epoch 751/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4303e-04 - val_loss: 1.7518e-04
Epoch 752/1000
32/32 [==============================] - 0s 154us/step - loss: 3.4263e-04 - val_loss: 1.7494e-04
Epoch 753/1000
32/32 [==============================] - 0s 141us/step - loss: 3.4223e-04 - val_loss: 1.7470e-04
Epoch 754/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4183e-04 - val_loss: 1.7446e-04
Epoch 755/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4144e-04 - val_loss: 1.7421e-04
Epoch 756/1000
32/32 [==============================] - 0s 155us/step - loss: 3.4104e-04 - val_loss: 1.7398e-04
Epoch 757/1000
32/32 [==============================] - 0s 160us/step - loss: 3.4065e-04 - val_loss: 1.7

Epoch 823/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1566e-04 - val_loss: 1.5911e-04
Epoch 824/1000
32/32 [==============================] - 0s 157us/step - loss: 3.1529e-04 - val_loss: 1.5890e-04
Epoch 825/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1494e-04 - val_loss: 1.5870e-04
Epoch 826/1000
32/32 [==============================] - 0s 157us/step - loss: 3.1457e-04 - val_loss: 1.5849e-04
Epoch 827/1000
32/32 [==============================] - 0s 136us/step - loss: 3.1421e-04 - val_loss: 1.5827e-04
Epoch 828/1000
32/32 [==============================] - 0s 125us/step - loss: 3.1385e-04 - val_loss: 1.5808e-04
Epoch 829/1000
32/32 [==============================] - 0s 154us/step - loss: 3.1350e-04 - val_loss: 1.5787e-04
Epoch 830/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1313e-04 - val_loss: 1.5767e-04
Epoch 831/1000
32/32 [==============================] - 0s 125us/step - loss: 3.1277e-04 - val_loss: 1.5

Epoch 897/1000
32/32 [==============================] - 0s 126us/step - loss: 2.9010e-04 - val_loss: 1.4478e-04
Epoch 898/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8977e-04 - val_loss: 1.4461e-04
Epoch 899/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8944e-04 - val_loss: 1.4443e-04
Epoch 900/1000
32/32 [==============================] - 0s 124us/step - loss: 2.8911e-04 - val_loss: 1.4425e-04
Epoch 901/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8878e-04 - val_loss: 1.4407e-04
Epoch 902/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8846e-04 - val_loss: 1.4389e-04
Epoch 903/1000
32/32 [==============================] - 0s 137us/step - loss: 2.8813e-04 - val_loss: 1.4371e-04
Epoch 904/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8781e-04 - val_loss: 1.4352e-04
Epoch 905/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8748e-04 - val_loss: 1.4

Epoch 971/1000
32/32 [==============================] - 0s 166us/step - loss: 2.6685e-04 - val_loss: 1.3218e-04
Epoch 972/1000
32/32 [==============================] - 0s 124us/step - loss: 2.6655e-04 - val_loss: 1.3202e-04
Epoch 973/1000
32/32 [==============================] - 0s 126us/step - loss: 2.6625e-04 - val_loss: 1.3186e-04
Epoch 974/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6596e-04 - val_loss: 1.3170e-04
Epoch 975/1000
32/32 [==============================] - 0s 144us/step - loss: 2.6566e-04 - val_loss: 1.3153e-04
Epoch 976/1000
32/32 [==============================] - 0s 157us/step - loss: 2.6536e-04 - val_loss: 1.3138e-04
Epoch 977/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6506e-04 - val_loss: 1.3121e-04
Epoch 978/1000
32/32 [==============================] - 0s 137us/step - loss: 2.6477e-04 - val_loss: 1.3107e-04
Epoch 979/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6447e-04 - val_loss: 1.3

[array([[-1.07478216e-01, -5.58431819e-02,  9.23008502e-01,
         -5.87950826e-01,  8.97495210e-01, -3.01526189e-02,
          3.88361424e-01, -1.01253033e-01, -8.32183063e-01,
         -1.19513474e-01],
        [ 3.74152027e-02, -8.65451038e-01,  3.45074326e-01,
          7.02221245e-02, -4.63429213e-01,  8.10471475e-01,
          1.02656221e+00,  7.70139595e-05, -4.87237424e-02,
          2.29491323e-01],
        [ 1.49718419e-01,  6.35132253e-01,  3.38352472e-01,
         -1.11239564e+00,  1.41564339e-01, -1.21264923e+00,
          6.95713520e-01,  3.55192670e-03, -1.32241797e+00,
          4.49131802e-03]], dtype=float32),
 array([ 0.7795621 , -0.5692419 ,  0.364034  ,  0.14495759, -0.400672  ,
         0.21324238, -0.5116837 , -0.6678646 , -0.02342039, -0.5632702 ],
       dtype=float32),
 array([[ 0.8497109 , -0.30677885, -0.8839    , -0.67220974, -0.61050785],
        [-0.20088618, -0.73185647,  0.4013172 , -0.02770892,  0.6377881 ],
        [-0.12236369, -0.22878511, -0.4070

In [69]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure4_4th.h5')

In [70]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_5(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_45 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_46 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 13ms/step - loss: 37.

Epoch 71/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0109 - val_loss: 0.0121
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0127 - val_loss: 0.0136
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0134 - val_loss: 0.0138
Epoch 74/1000
32/32 [==============================] - 0s 132us/step - loss: 0.0128 - val_loss: 0.0128
Epoch 75/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0112 - val_loss: 0.0112
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0094 - val_loss: 0.0093
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0079 - val_loss: 0.0073
Epoch 78/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0071 - val_loss: 0.0058
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0071 - val_loss: 0.0047
Epoch 80/1000
32/32 [==============================] - 0s 130us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0036 - val_loss: 0.0041
Epoch 152/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0036 - val_loss: 0.0041
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0041
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0035 - val_loss: 0.0040
Epoch 155/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0035 - val_loss: 0.0040
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0040
Epoch 157/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0034 - val_loss: 0.0040
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0040
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0040
Epoch 160/1000
32/32 [==============================] - 0s 124us

Epoch 230/1000
32/32 [==============================] - 0s 142us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 231/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 232/1000
32/32 [==============================] - 0s 167us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 233/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 237/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0020 - val_loss: 0.0030
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 0.0029
Epoch 239/1000
32/32 [==============================] - 0s 142us

Epoch 309/1000
32/32 [==============================] - 0s 135us/step - loss: 0.0014 - val_loss: 0.0022
Epoch 310/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 315/1000
32/32 [==============================] - 0s 159us/step - loss: 0.0013 - val_loss: 0.0021
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0021
Epoch 317/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0021
Epoch 318/1000
32/32 [==============================] - 0s 155us

...

Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1957e-04 - val_loss: 4.2919e-04
Epoch 762/1000
32/32 [==============================] - 0s 131us/step - loss: 2.1882e-04 - val_loss: 4.2795e-04
Epoch 763/1000
32/32 [==============================] - 0s 155us/step - loss: 2.1808e-04 - val_loss: 4.2671e-04
Epoch 764/1000
32/32 [==============================] - 0s 149us/step - loss: 2.1735e-04 - val_loss: 4.2550e-04
Epoch 765/1000
32/32 [==============================] - 0s 187us/step - loss: 2.1661e-04 - val_loss: 4.2428e-04
Epoch 766/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1588e-04 - val_loss: 4.2304e-04
Epoch 767/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1515e-04 - val_loss: 4.2184e-04
Epoch 768/1000
32/32 [==============================] - 0s 131us/step - loss: 2.1442e-04 - val_loss: 4.2066e-04
Epoch 769/1000
32/32 [==============================] - 0s 148us/step - loss: 2.1370e-04 - val_loss: 4.1

Epoch 835/1000
32/32 [==============================] - 0s 186us/step - loss: 1.7212e-04 - val_loss: 3.4892e-04
Epoch 836/1000
32/32 [==============================] - 0s 155us/step - loss: 1.7158e-04 - val_loss: 3.4797e-04
Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7103e-04 - val_loss: 3.4703e-04
Epoch 838/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7049e-04 - val_loss: 3.4609e-04
Epoch 839/1000
32/32 [==============================] - 0s 187us/step - loss: 1.6995e-04 - val_loss: 3.4515e-04
Epoch 840/1000
32/32 [==============================] - 0s 155us/step - loss: 1.6941e-04 - val_loss: 3.4422e-04
Epoch 841/1000
32/32 [==============================] - 0s 154us/step - loss: 1.6888e-04 - val_loss: 3.4330e-04
Epoch 842/1000
32/32 [==============================] - 0s 186us/step - loss: 1.6835e-04 - val_loss: 3.4239e-04
Epoch 843/1000
32/32 [==============================] - 0s 187us/step - loss: 1.6782e-04 - val_loss: 3.4

Epoch 909/1000
32/32 [==============================] - 0s 187us/step - loss: 1.3720e-04 - val_loss: 2.8699e-04
Epoch 910/1000
32/32 [==============================] - 0s 187us/step - loss: 1.3680e-04 - val_loss: 2.8628e-04
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3640e-04 - val_loss: 2.8553e-04
Epoch 912/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3600e-04 - val_loss: 2.8481e-04
Epoch 913/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3560e-04 - val_loss: 2.8409e-04
Epoch 914/1000
32/32 [==============================] - 0s 155us/step - loss: 1.3520e-04 - val_loss: 2.8336e-04
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3480e-04 - val_loss: 2.8263e-04
Epoch 916/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3441e-04 - val_loss: 2.8192e-04
Epoch 917/1000
32/32 [==============================] - 0s 187us/step - loss: 1.3402e-04 - val_loss: 2.8

Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1136e-04 - val_loss: 2.3870e-04
Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1106e-04 - val_loss: 2.3814e-04
Epoch 985/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1077e-04 - val_loss: 2.3756e-04
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1047e-04 - val_loss: 2.3701e-04
Epoch 987/1000
32/32 [==============================] - 0s 125us/step - loss: 1.1018e-04 - val_loss: 2.3645e-04
Epoch 988/1000
32/32 [==============================] - 0s 124us/step - loss: 1.0988e-04 - val_loss: 2.3586e-04
Epoch 989/1000
32/32 [==============================] - 0s 124us/step - loss: 1.0959e-04 - val_loss: 2.3531e-04
Epoch 990/1000
32/32 [==============================] - 0s 155us/step - loss: 1.0930e-04 - val_loss: 2.3473e-04
Epoch 991/1000
32/32 [==============================] - 0s 142us/step - loss: 1.0900e-04 - val_loss: 2.3

[array([[ 0.7822377 ,  0.3867028 , -0.03218643,  1.1586654 , -0.44858235,
         -0.7472017 , -0.22837785, -0.91857594,  1.2088966 ,  0.5375606 ],
        [-1.0978749 ,  0.3386378 ,  0.22387356,  0.23045193,  1.0139182 ,
          0.3385366 , -1.0568665 ,  0.15739894,  0.30679914, -0.7370963 ],
        [ 0.02880979,  1.1188565 ,  0.5663107 ,  0.23344311, -0.17907685,
         -0.6992562 , -0.69969445,  1.1280634 , -0.36329   ,  0.65127325]],
       dtype=float32),
 array([ 0.4452581 ,  0.38921785,  0.278998  , -0.49206334, -0.09413301,
        -0.3220109 ,  0.37730265, -0.27143547,  0.43291664, -0.5959225 ],
       dtype=float32),
 array([[-0.15462038, -0.07746185,  0.272846  , -0.08471205,  0.37236002,
         -0.03618853,  0.08720662, -0.22447324,  0.07351431,  0.29435465],
        [ 0.47560188,  0.23134413,  0.5648685 , -0.4773119 ,  0.17680456,
          0.12190033,  0.12044638,  0.7221592 , -0.44361025, -0.44269878],
        [ 0.09781466,  0.32607216,  0.47720617, -0.23628308, 

In [71]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure5_4th.h5')

In [72]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_6(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_47 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_48 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 35.

Epoch 71/1000
32/32 [==============================] - 0s 163us/step - loss: 0.0130 - val_loss: 0.0133
Epoch 72/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0145 - val_loss: 0.0132
Epoch 73/1000
32/32 [==============================] - 0s 160us/step - loss: 0.0149 - val_loss: 0.0121
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0141 - val_loss: 0.0103
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0123 - val_loss: 0.0083
Epoch 76/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0103 - val_loss: 0.0065
Epoch 77/1000
32/32 [==============================] - 0s 144us/step - loss: 0.0085 - val_loss: 0.0054
Epoch 78/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0075 - val_loss: 0.0048
Epoch 79/1000
32/32 [==============================] - 0s 145us/step - loss: 0.0074 - val_loss: 0.0047
Epoch 80/1000
32/32 [==============================] - 0s 124us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0043
Epoch 152/1000
32/32 [==============================] - 0s 152us/step - loss: 0.0039 - val_loss: 0.0042
Epoch 153/1000
32/32 [==============================] - 0s 137us/step - loss: 0.0039 - val_loss: 0.0042
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0041
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0041
Epoch 156/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0038 - val_loss: 0.0040
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0040
Epoch 158/1000
32/32 [==============================] - 0s 177us/step - loss: 0.0038 - val_loss: 0.0039
Epoch 159/1000
32/32 [==============================] - 0s 169us/step - loss: 0.0038 - val_loss: 0.0039
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0026 - val_loss: 0.0021
Epoch 231/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0026 - val_loss: 0.0021
Epoch 232/1000
32/32 [==============================] - 0s 122us/step - loss: 0.0026 - val_loss: 0.0021
Epoch 233/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0026 - val_loss: 0.0021
Epoch 234/1000
32/32 [==============================] - 0s 147us/step - loss: 0.0026 - val_loss: 0.0020
Epoch 235/1000
32/32 [==============================] - 0s 130us/step - loss: 0.0026 - val_loss: 0.0020
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0020
Epoch 237/1000
32/32 [==============================] - 0s 148us/step - loss: 0.0025 - val_loss: 0.0020
Epoch 238/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0025 - val_loss: 0.0020
Epoch 239/1000
32/32 [==============================] - 0s 125us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 310/1000
32/32 [==============================] - 0s 133us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 312/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 313/1000
32/32 [==============================] - 0s 120us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 314/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 315/1000
32/32 [==============================] - 0s 147us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 316/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 317/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0010
Epoch 318/1000
32/32 [==============================] - 0s 151us

...

Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3981e-04 - val_loss: 2.9145e-05
Epoch 759/1000
32/32 [==============================] - 0s 187us/step - loss: 4.3863e-04 - val_loss: 2.8948e-05
Epoch 760/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3744e-04 - val_loss: 2.8751e-05
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3626e-04 - val_loss: 2.8556e-05
Epoch 762/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3509e-04 - val_loss: 2.8363e-05
Epoch 763/1000
32/32 [==============================] - 0s 187us/step - loss: 4.3392e-04 - val_loss: 2.8172e-05
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3276e-04 - val_loss: 2.7984e-05
Epoch 765/1000
32/32 [==============================] - 0s 148us/step - loss: 4.3160e-04 - val_loss: 2.7796e-05
Epoch 766/1000
32/32 [==============================] - 0s 151us/step - loss: 4.3043e-04 - val_loss: 2.7

Epoch 832/1000
32/32 [==============================] - 0s 156us/step - loss: 3.6082e-04 - val_loss: 1.8412e-05
Epoch 833/1000
32/32 [==============================] - 0s 187us/step - loss: 3.5986e-04 - val_loss: 1.8307e-05
Epoch 834/1000
32/32 [==============================] - 0s 187us/step - loss: 3.5891e-04 - val_loss: 1.8204e-05
Epoch 835/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5795e-04 - val_loss: 1.8105e-05
Epoch 836/1000
32/32 [==============================] - 0s 187us/step - loss: 3.5701e-04 - val_loss: 1.8005e-05
Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5606e-04 - val_loss: 1.7904e-05
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5511e-04 - val_loss: 1.7805e-05
Epoch 839/1000
32/32 [==============================] - 0s 180us/step - loss: 3.5418e-04 - val_loss: 1.7709e-05
Epoch 840/1000
32/32 [==============================] - 0s 143us/step - loss: 3.5324e-04 - val_loss: 1.7

Epoch 906/1000
32/32 [==============================] - 0s 124us/step - loss: 2.9665e-04 - val_loss: 1.2762e-05
Epoch 907/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9588e-04 - val_loss: 1.2705e-05
Epoch 908/1000
32/32 [==============================] - 0s 149us/step - loss: 2.9509e-04 - val_loss: 1.2650e-05
Epoch 909/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9432e-04 - val_loss: 1.2597e-05
Epoch 910/1000
32/32 [==============================] - 0s 167us/step - loss: 2.9353e-04 - val_loss: 1.2543e-05
Epoch 911/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9276e-04 - val_loss: 1.2490e-05
Epoch 912/1000
32/32 [==============================] - 0s 175us/step - loss: 2.9199e-04 - val_loss: 1.2435e-05
Epoch 913/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9122e-04 - val_loss: 1.2385e-05
Epoch 914/1000
32/32 [==============================] - 0s 125us/step - loss: 2.9045e-04 - val_loss: 1.2

Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4404e-04 - val_loss: 9.6849e-06
Epoch 981/1000
32/32 [==============================] - 0s 187us/step - loss: 2.4340e-04 - val_loss: 9.6542e-06
Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4276e-04 - val_loss: 9.6247e-06
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4212e-04 - val_loss: 9.5966e-06
Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4148e-04 - val_loss: 9.5633e-06
Epoch 985/1000
32/32 [==============================] - 0s 114us/step - loss: 2.4085e-04 - val_loss: 9.5365e-06
Epoch 986/1000
32/32 [==============================] - 0s 131us/step - loss: 2.4021e-04 - val_loss: 9.5076e-06
Epoch 987/1000
32/32 [==============================] - 0s 187us/step - loss: 2.3958e-04 - val_loss: 9.4778e-06
Epoch 988/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3895e-04 - val_loss: 9.4

[array([[ 0.52988815,  0.32709607, -0.765472  , -0.4203772 ,  0.20353556,
         -0.22286743, -0.12472472, -0.37427777, -0.47253963,  0.01451027],
        [-0.346148  ,  0.05517873,  0.3676369 ,  0.07804321, -0.45801392,
          0.30699843, -0.7116298 , -0.20733434, -0.22980839,  0.5276024 ],
        [ 1.0075349 , -0.6044144 , -0.31765264,  0.5946219 ,  0.2833864 ,
          0.5943915 , -0.7562337 , -0.13939127, -1.1358294 , -0.8597611 ]],
       dtype=float32),
 array([ 0.29248744, -0.32539675,  0.23257385, -0.01538891, -0.5564692 ,
        -0.40124053,  0.28376395, -0.03583689, -0.38993198,  0.37683728],
       dtype=float32),
 array([[-0.4980242 ,  0.40676212, -0.3495227 ,  0.37335292,  0.21856794,
         -0.24666992,  0.5486869 ,  0.43239108,  0.1732213 ,  0.4514456 ,
          0.1617751 ,  0.30438593, -0.54482543,  0.41437465,  0.41851136],
        [-0.01545042,  0.30963987,  0.05096102, -0.4263212 , -0.01367264,
          0.36587614, -0.38234356,  0.19663402,  0.10444505,  

In [73]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure6_4th.h5')